
# Joule Heating Mean, Variance, Standard Variation per Bin

## Introduction
The purpose of this software is to calculate the mean, variance and standard deviation of Joule Heating for each
areas of interest, during a certain event.
We call these areas 'bins'.
The bin boundaries are defined by ranges of:
1. Magnetic Local Time (MLT)
2. Magnetic Latitude (MagLat)
3. Altitude
4. Geomagnetic Kp index


## Data
The software reads several TIEGCM NetCDF files. These files contain Joule Heating information about every region around the Earth associated with the corresponding MLT, MagLat, Altitude and Kp index, during the chosen event.
The result data are stored in order to be ploted easier without the time intensive calculation.
The execution and ploting is separated in regions to make calculations easier to handle and plot more clear.


## Algorithm Description
A) For calculating statistics for the while TIEGCM data of the satellite's lifetime:
     - for every point descibed in each of the NetCDF files:
        - check if the point belongs to any of the pre-defined bins.
        - if it does, then assign the Joule Heating value of this point to the correct bin.
     - for each bin calculate the mean, variance, standard deviation of Joule Heating.

B) For calculating statistics for the whole orbit of the satellite:
    - For every satellite position check if the satellite position lies inside some bin:
		1. read Altitude, Magnetic-Latitude, Magnetic-Local-Time.
		2. Check if the above values lie inside the ranges of a bin.
	   	   If they do then we have to check the Kp-value following the next step. 		   
		4. Kp-value is stored in a TIEGCM file. There are many TIEGCM files separated by time.
		   Read the time of the satellite position and locate the corresponding TIEGCM file.
		5. Read the Kp-value according to the current time.
		6. Now we can check if the satellite position really lies inside a bin.
		   If it does, then locate the 8 closest points of the TIEGCM grid and go to next step.
		7. Read the usefull values from the 8 TIEGCM points, interpolate them and calculate Joule Heating.
		8. Assign the Joule Heating value to the correct bin.


## Plots
Joule Heating mean and standard deviation values for each bin are plotted against MLT, MagLat, Altitude in 3 different plots. The mean is displayed as a horizontal line and the standard deviation as a vertical line. Dots represent instances of measurements. Not all the measurements are displayed because of their vast number. The same values are plotted again in different plots separated by the Kp-range of each bin.
There is also a plot of MagLat against Altitude where dots represent a measuremnt and their colors correspond to their Joule Heating value.
There is available a Joule Heating distribution plot which can display fiiting functions along the data.



In [17]:
import sys
sys.path.insert(1, '../../SourceCode/')
import DaedalusGlobals as DaedalusGlobals
import Conversions as Conversions

import os
from os import path
import statistics
import random 
from scipy.optimize import curve_fit

import csv
import glob
import math
import time
from datetime import datetime
from datetime import timezone
from dateutil.relativedelta import relativedelta
import calendar
import numpy as np 
import pandas as pd
import ipywidgets as w
from netCDF4 import Dataset 
from numba import cuda

import plotly
import chart_studio.plotly as py 
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.cm
import matplotlib.pyplot as matplt


# colors used at plotting
MyColors = ["#217ca3", "#e29930", "#919636", "#af1c1c", "#e7552c", "#1b4b5a", "#e4535e", "#aebd38", "#ffbb00", "#2c7873"]

# GUI elements with global scope
style1 = {'description_width':'170px'}
layout1 = {'width':'780px'}
style2 = {'description_width':'95px'}
layout2 = {'width':'160px'}
OrbitPreviewImage = w.Image( format='png', visible=False )
OrbitPreviewImage.layout.visibility = 'hidden'
ExecutionTitle_Text = w.Text(value="", description='Execution title:', style=style1, layout=layout1)
ExecutionDescr_Text = w.Text(value="", description='Execution description:', style=style1, layout=layout1)
Warning_HTML = w.HTML( value ="", color="Red", visible=False )
tiegcmFolder_Dropdown    = w.Dropdown( options=[DaedalusGlobals.TIEGCM_Files_Path+"TIEGCM_Lifetime_2015_to_2018_JH_QD/"], description='TIEGCM files: ', style=style1, layout=layout1)
BinGroups_Dropdown       = w.Dropdown( options=["AEM", "AFM", "AEE", "AED", "EEJ", "EPB", "SQ", "CF", "PCF"], description='Area of study: ', style=style1, layout=layout1)
OrbitFilename_Dropdown   = w.Dropdown( options=sorted(glob.glob(DaedalusGlobals.Orbit_Files_Path + "DAED_ORB_Lifetime*.csv")), description='Along orbit filename: ', style=style1, layout=layout1)
SavedFilenames_Dropdown  = w.Dropdown( options=sorted(glob.glob(DaedalusGlobals.CoverageResults_Files_Path + "*.JHperBinResults.txt")),  description='', style=style1, layout=layout1)
SavedFilenames2_Dropdown = w.Dropdown( options=sorted(glob.glob(DaedalusGlobals.CoverageResults_Files_Path + "*.JHperBinResults.txt")),  description='', style=style1, layout=layout1)
Plot_JHvsMagLat_Checkbox       = w.Checkbox(value=True, description="Plot JH vs Magnetic Latitude", style=style1, layout=layout1 )
Plot_JHvsMLT_Checkbox          = w.Checkbox(value=True, description="Plot JH vs Magnetic Local Time", style=style1, layout=layout1 )
Plot_JHvsAltitude_Checkbox     = w.Checkbox(value=True, description="Plot JH vs Altitude", style=style1, layout=layout1 )
Plot_AltitudeVsMagLat_Checkbox = w.Checkbox(value=True, description="Plot Altitude vs Magnetic Latitude", style=style1, layout=layout1 )
Plot_JHdistribution_Checkbox   = w.Checkbox(value=True, description="Plot JH distribution per bin", style=style1, layout=layout1 )
RegressionOptions_Dropdown     = w.Dropdown( options=["None", "Polynomial - degree 1", "Polynomial - degree 2", "Polynomial - degree 3", "Polynomial - degree 4", "Polynomial - degree 5", "Polynomial - degree 6", "Power law", "Logarithmic", "Euler", "Maxwell"],  description='Regression Analysis', style=style1, layout=layout1)


# Properties of the current calculation
CALCULATIONS_Title = ""
CALCULATIONS_Description =""
CALCULATIONS_RegionName = ""
CALCULATIONS_OrbitFilename = ""
CALCULATIONS_ResultsFilename = ""
CALCULATIONS_DataPath = ""
CALCULATIONS_ExecutionDuration = 0
all_JH_values       = list() # values for ploting (Y axis)
all_MagLat_values   = list() # values for ploting (X axis)
all_MLT_values      = list() # values for ploting (X axis)
all_Altitude_values = list() # values for ploting (X axis)

# utility: converts a number to its 2-digit string representation
def num_to_2digit_str( n ):
    s = str(n)
    if len(s) == 1:
        s = '0' + s
    return s

# utility: takes a string containing numbers and places spaces instead of the leading zeros 
def ConvertLeadingZerosToSpaces( str ):
    result = ""
    leading_zone = True
    for c in str:
        if leading_zone:
            if c == '0':
                result = result + ' '
            else:
                result = result + c
                leading_zone = False
        else:
            result = result + c
    if result.strip().startswith('.')  and  result.startswith(' '): result = result[:result.rfind(' ')] + '0' + result.strip()
    if result.strip() == "": result = result[ :-1 ] + '0'
    if (result.startswith('.')) : result = '0' + result            
    return result

# Parses a string representing a date and returns a corresponding datetime object. Example: Jan 01 2015 00:01:10.000000000
def parseDaedalusDate( dateString ):
    result = None
    try:
        result = datetime.strptime(dateString[0:24], '%b %d %Y %H:%M:%S.%f')
    except:
        try:
            result = datetime.strptime(dateString, '%b %d %Y %H:%M:%S.%f')
        except:
            try:
                result = datetime.strptime(dateString, '%d %b %Y %H:%M:%S.%f')
            except:
                result = None
    return result
        

# utility: returns a color of a colormap as list of r,g,b,a values representing a value inside a range
def getColor( Value, minValue, maxValue, ColormapName ):
    cmap = matplotlib.cm.get_cmap( ColormapName )
    norm = matplotlib.colors.Normalize(vmin=minValue, vmax=maxValue)
    rgba = cmap( norm(Value) )
    s = "rgba" + str(rgba) 
    return s

# Define a class which can describe a bin
class Bin:
    ID             = ""
    Description    = ""
    MLT_min        = 0 # Magnetic Local Time (hour & min of the 24-hour day) (string)
    MLT_max        = 0 # Magnetic Local Time (hour & min of the 24-hour day) (string)
    MagLat_min     = 0 # Magnetic Latitude (degrees)
    MagLat_max     = 0 # Magnetic Latitude (degrees)
    Altitude_min   = 0 # Satellite's Altitude measured from Earth's surface (km)
    Altitude_max   = 0 # Satellite's Altitude measured from Earth's surface (km)
    Kp_min         = 0 #
    Kp_max         = 0 #
    NumOfBins      = 0 # How many parts will the Altitude range be splitted in
    CumulativeTime = 0 # (sec)
    DesirableCumulativeTime = 0 # (sec)
    JH_min      = 99999 # the minimum JH value inside the bin
    JH_max      = 0     # the maximum JH value inside the bin
    JH_mean     = 0 # the mean JH value inside the bin
    JH_variance = 0 # the variance of JH value inside the bin (variance = (1/(N-1)) * Sum{1->N}(X-MeanVariance)^2  )
    # Data:
    JH_values         = list() # here will be stored all Joule Heating values in order to calculate the variance at the end
    JH_distribution   = list() # the item 0 holds the number of points which have 0<JH<JH_max/100 etc
    MagLat_values     = list() #  these values correspond to the JH_values
    MLT_values        = list() #  these values correspond to the JH_values
    Altitude_values   = list() #  these values correspond to the JH_values
    
    
    
    def __init__(self, ID, Description, MLT_min, MLT_max, MagLat_min, MagLat_max, Altitude_min, Altitude_max, Kp_min, Kp_max, DesirableCumulativeTime):
        self.ID             = ID
        self.Description    = Description
        self.MLT_min        = MLT_min 
        self.MLT_max        = MLT_max
        self.MagLat_min     = MagLat_min
        self.MagLat_max     = MagLat_max
        self.Altitude_min   = Altitude_min
        self.Altitude_max   = Altitude_max
        self.Kp_min         = Kp_min
        self.Kp_max         = Kp_max
        self.DesirableCumulativeTime = DesirableCumulativeTime
        self.JH_values       = list()
        self.JH_distribution = [0] * 100
        self.MagLat_values   = list()
        self.MLT_values      = list()
        self.Altitude_values = list()

    def reset(self):
        self.JH_min      = 99999
        self.JH_mean     = 0
        self.JH_variance = 0
        self.JH_values       = list()
        self.MagLat_values   = list()
        self.MLT_values      = list()
        self.Altitude_values = list()
        
        
    def getInfo(self):
        s  = self.ID.ljust(8, ' ') + ": "
        s += "{:02.0f}".format(self.MLT_min)      + "<MLT<="    + "{:02.0f}".format(self.MLT_max)      + " "
        s += "{:03.0f}".format(self.MagLat_min)   + "<MagLat<=" + "{:03.0f}".format(self.MagLat_max)   + " "
        s += "{:03.0f}".format(self.Altitude_min) + "<Alt<="    + "{:03.0f}".format(self.Altitude_max) + " "
        s += str(self.Kp_min)             + "<Kp<="     + str(self.Kp_max)       + " "
        if self.JH_min == 99999:
            s += " JHmin=" + "         "
        else:
            s += " JHmin=" + "{:.3e}".format(self.JH_min) #ConvertLeadingZerosToSpaces( "{:09.3f}".format(self.JH_min) )
        s += " JHmean=" + "{:.3e}".format(self.JH_mean) #ConvertLeadingZerosToSpaces( "{:09.3f}".format(self.JH_mean) )
        s += " JHvariance=" + "{:.3e}".format(self.JH_variance) #ConvertLeadingZerosToSpaces( "{:09.3f}".format(self.JH_variance) )
        ##
        str_JH = ""
        for i in range(0, len(self.JH_values) ):            
            str_JH += str( self.JH_values[i] )
            if i < len(self.JH_values)-1: str_JH += ','
        s += " JH_values=" + str_JH # ''.join(str(e) for e in self.JH_values)
        ##
        return s
    
    def printMe(self):
        print( self.getInfo()[:220] )


Bins = list() # this list holds the definitions of all bins
def InitializeBins():
    global Bins
    Bins = list()
    #                ID        Description                          MLT      MagLat    Altitude                Kp       DesiredTime(sec)
    Bins.append( Bin("AEM_L1", "Auroral E region, midnight sector", 22, 2,   60, 75,   115, 120,               0, 2,   50*60 ) )
    Bins.append( Bin("AEM_L2", "Auroral E region, midnight sector", 22, 2,   60, 75,   120, 125,               0, 2,   50*60 ) )
    Bins.append( Bin("AEM_L3", "Auroral E region, midnight sector", 22, 2,   60, 75,   125, 130,               0, 2,   50*60 ) )
    Bins.append( Bin("AEM_L4", "Auroral E region, midnight sector", 22, 2,   60, 75,   130, 135,               0, 2,   50*60 ) )
    Bins.append( Bin("AEM_L5", "Auroral E region, midnight sector", 22, 2,   60, 75,   135, 140,               0, 2,   50*60 ) )
    Bins.append( Bin("AEM_M1", "Auroral E region, midnight sector", 22, 2,   60, 75,   115,       123.33333,   2, 4,   30*60 ) )
    Bins.append( Bin("AEM_M2", "Auroral E region, midnight sector", 22, 2,   60, 75,   123.33333, 131.66666,   2, 4,   30*60 ) )
    Bins.append( Bin("AEM_M3", "Auroral E region, midnight sector", 22, 2,   60, 75,   131.66666, 140,         2, 4,   30*60 ) )
    Bins.append( Bin("AEM_H1", "Auroral E region, midnight sector", 22, 2,   60, 75,   115, 140,               4, 9,   20*60 ) )
    
    Bins.append( Bin("AFM_L1", "Auroral F region, midnight sector", 22, 2,   60, 75,   140, 185,               0, 2,   50*60 ) )
    Bins.append( Bin("AFM_L2", "Auroral F region, midnight sector", 22, 2,   60, 75,   185, 230,               0, 2,   50*60 ) )
    Bins.append( Bin("AFM_L3", "Auroral F region, midnight sector", 22, 2,   60, 75,   230, 275,               0, 2,   50*60 ) )
    Bins.append( Bin("AFM_L4", "Auroral F region, midnight sector", 22, 2,   60, 75,   275, 320,               0, 2,   50*60 ) )
    Bins.append( Bin("AFM_L5", "Auroral F region, midnight sector", 22, 2,   60, 75,   320, 365,               0, 2,   50*60 ) )
    Bins.append( Bin("AFM_L6", "Auroral F region, midnight sector", 22, 2,   60, 75,   365, 410,               0, 2,   50*60 ) )
    Bins.append( Bin("AFM_L7", "Auroral F region, midnight sector", 22, 2,   60, 75,   410, 455,               0, 2,   50*60 ) )
    Bins.append( Bin("AFM_L8", "Auroral F region, midnight sector", 22, 2,   60, 75,   455, 500,               0, 2,   50*60 ) )
    Bins.append( Bin("AFM_M1", "Auroral F region, midnight sector", 22, 2,   60, 75,   140, 230,               2, 4,   30*60 ) )
    Bins.append( Bin("AFM_M2", "Auroral F region, midnight sector", 22, 2,   60, 75,   230, 320,               2, 4,   30*60 ) )
    Bins.append( Bin("AFM_M3", "Auroral F region, midnight sector", 22, 2,   60, 75,   320, 410,               2, 4,   30*60 ) )
    Bins.append( Bin("AFM_M4", "Auroral F region, midnight sector", 22, 2,   60, 75,   410, 500,               2, 4,   30*60 ) )
    Bins.append( Bin("AFM_H1", "Auroral F region, midnight sector", 22, 2,   60, 75,   140, 260,               4, 9,   20*60 ) )
    Bins.append( Bin("AFM_H2", "Auroral F region, midnight sector", 22, 2,   60, 75,   260, 380,               4, 9,   20*60 ) )
    Bins.append( Bin("AFM_H3", "Auroral F region, midnight sector", 22, 2,   60, 75,   380, 500,               4, 9,   20*60 ) )
    
    Bins.append( Bin("AEE_L1", "Auroral E region, evening sector",  16, 20,  60, 75,   115, 120,               0, 2,   50*60 ) )
    Bins.append( Bin("AEE_L2", "Auroral E region, evening sector",  16, 20,  60, 75,   120, 125,               0, 2,   50*60 ) )
    Bins.append( Bin("AEE_L3", "Auroral E region, evening sector",  16, 20,  60, 75,   125, 130,               0, 2,   50*60 ) )
    Bins.append( Bin("AEE_L4", "Auroral E region, evening sector",  16, 20,  60, 75,   130, 135,               0, 2,   50*60 ) )
    Bins.append( Bin("AEE_L5", "Auroral E region, evening sector",  16, 20,  60, 75,   135, 140,               0, 2,   50*60 ) )
    Bins.append( Bin("AEE_M1", "Auroral E region, evening sector",  16, 20,  60, 75,   115,       123.33333,   2, 4,   30*60 ) )
    Bins.append( Bin("AEE_M2", "Auroral E region, evening sector",  16, 20,  60, 75,   123.33333, 131.66666,   2, 4,   30*60 ) )
    Bins.append( Bin("AEE_M3", "Auroral E region, evening sector",  16, 20,  60, 75,   131.66666, 140,         2, 4,   30*60 ) )
    Bins.append( Bin("AEE_H1", "Auroral E region, evening sector",  16, 20,  60, 75,   115, 140,               4, 9,   20*60 ) )
    
    Bins.append( Bin("AED_L1", "Auroral E region, dawn sector",     4, 8,  60, 75,   115, 120,                 0, 2,   50*60 ) )
    Bins.append( Bin("AED_L2", "Auroral E region, dawn sector",     4, 8,  60, 75,   120, 125,                 0, 2,   50*60 ) )
    Bins.append( Bin("AED_L3", "Auroral E region, dawn sector",     4, 8,  60, 75,   125, 130,                 0, 2,   50*60 ) )
    Bins.append( Bin("AED_L4", "Auroral E region, dawn sector",     4, 8,  60, 75,   130, 135,                 0, 2,   50*60 ) )
    Bins.append( Bin("AED_L5", "Auroral E region, dawn sector",     4, 8,  60, 75,   135, 140,                 0, 2,   50*60 ) )
    Bins.append( Bin("AED_M1", "Auroral E region, dawn sector",     4, 8,  60, 75,   115,       123.33333,     2, 4,   30*60 ) )
    Bins.append( Bin("AED_M2", "Auroral E region, dawn sector",     4, 8,  60, 75,   123.33333, 131.66666,     2, 4,   30*60 ) )
    Bins.append( Bin("AED_M3", "Auroral E region, dawn sector",     4, 8,  60, 75,   131.66666, 140,           2, 4,   30*60 ) )
    Bins.append( Bin("AED_H1", "Auroral E region, dawn sector",     4, 8,  60, 75,   115, 140,                 4, 9,   20*60 ) )
    
    Bins.append( Bin("EEJ_A1", "Equatorial E-region",             10, 13,  -7,  7,   115,       123.33333,     0, 9,   10*60 ) )
    Bins.append( Bin("EEJ_A2", "Equatorial E-region",             10, 13,  -7,  7,   123.33333, 131.66666,     0, 9,   10*60 ) )
    Bins.append( Bin("EEJ_A3", "Equatorial E-region",             10, 13,  -7,  7,   131.66666, 140,           0, 9,   10*60 ) )
    
    Bins.append( Bin("EPB_A1", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   140, 185,                  0, 9,   150*60 ) )
    Bins.append( Bin("EPB_A2", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   185, 230,                  0, 9,   150*60 ) )
    Bins.append( Bin("EPB_A3", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   230, 275,                  0, 9,   150*60 ) )
    Bins.append( Bin("EPB_A4", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   275, 320,                  0, 9,   150*60 ) )
    Bins.append( Bin("EPB_A5", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   320, 365,                  0, 9,   150*60 ) )
    Bins.append( Bin("EPB_A6", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   365, 410,                  0, 9,   150*60 ) )
    Bins.append( Bin("EPB_A7", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   410, 455,                  0, 9,   150*60 ) )
    Bins.append( Bin("EPB_A8", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   455, 500,                  0, 9,   150*60 ) )
    
    Bins.append( Bin("SQ_A1",  "Sq & midlat F region currents",    6, 19, -60, 60,   140, 185,                  0, 3,   150*60 ) )
    Bins.append( Bin("SQ_A2",  "Sq & midlat F region currents",    6, 19, -60, 60,   185, 230,                  0, 3,   150*60 ) )
    Bins.append( Bin("SQ_A3",  "Sq & midlat F region currents",    6, 19, -60, 60,   230, 275,                  0, 3,   150*60 ) )
    Bins.append( Bin("SQ_A4",  "Sq & midlat F region currents",    6, 19, -60, 60,   275, 320,                  0, 3,   150*60 ) )
    Bins.append( Bin("SQ_A5",  "Sq & midlat F region currents",    6, 19, -60, 60,   320, 365,                  0, 3,   150*60 ) )
    Bins.append( Bin("SQ_A6",  "Sq & midlat F region currents",    6, 19, -60, 60,   365, 410,                  0, 3,   150*60 ) )
    Bins.append( Bin("SQ_A7",  "Sq & midlat F region currents",    6, 19, -60, 60,   410, 455,                  0, 3,   150*60 ) )
    Bins.append( Bin("SQ_A8",  "Sq & midlat F region currents",    6, 19, -60, 60,   455, 500,                  0, 3,   150*60 ) )
    
    Bins.append( Bin("CF_L1", "Dayside Cusp F-region",            10, 14,   70,  80,   140, 185,                0, 2,   50*60 ) )
    Bins.append( Bin("CF_L2", "Dayside Cusp F-region",            10, 14,   70,  80,   185, 230,                0, 2,   50*60 ) )
    Bins.append( Bin("CF_L3", "Dayside Cusp F-region",            10, 14,   70,  80,   230, 275,                0, 2,   50*60 ) )
    Bins.append( Bin("CF_L4", "Dayside Cusp F-region",            10, 14,   70,  80,   275, 320,                0, 2,   50*60 ) )
    Bins.append( Bin("CF_L5", "Dayside Cusp F-region",            10, 14,   70,  80,   320, 365,                0, 2,   50*60 ) )
    Bins.append( Bin("CF_L6", "Dayside Cusp F-region",            10, 14,   70,  80,   365, 410,                0, 2,   50*60 ) )
    Bins.append( Bin("CF_L7", "Dayside Cusp F-region",            10, 14,   70,  80,   410, 455,                0, 2,   50*60 ) )
    Bins.append( Bin("CF_L8", "Dayside Cusp F-region",            10, 14,   70,  80,   455, 500,                0, 2,   50*60 ) )
    Bins.append( Bin("CF_M1", "Dayside Cusp F-region",            10, 14,   70,  80,   140, 230,               2, 4,   30*60 ) )
    Bins.append( Bin("CF_M2", "Dayside Cusp F-region",            10, 14,   70,  80,   230, 320,               2, 4,   30*60 ) )
    Bins.append( Bin("CF_M3", "Dayside Cusp F-region",            10, 14,   70,  80,   320, 410,               2, 4,   30*60 ) )
    Bins.append( Bin("CF_M4", "Dayside Cusp F-region",            10, 14,   70,  80,   410, 500,               2, 4,   30*60 ) )
    Bins.append( Bin("CF_H1", "Dayside Cusp F-region",            10, 14,   70,  80,   140, 260,               4, 9,   20*60 ) )
    Bins.append( Bin("CF_H2", "Dayside Cusp F-region",            10, 14,   70,  80,   260, 380,               4, 9,   20*60 ) )
    Bins.append( Bin("CF_H3", "Dayside Cusp F-region",            10, 14,   70,  80,   380, 500,               4, 9,   20*60 ) )
    
    Bins.append( Bin("PCF_L1", "Polar cap F-region",              14, 10,   70,  90,   140, 185,               0, 2,   50*60 ) )
    Bins.append( Bin("PCF_L2", "Polar cap F-region",              14, 10,   70,  90,   185, 230,               0, 2,   50*60 ) )
    Bins.append( Bin("PCF_L3", "Polar cap F-region",              14, 10,   70,  90,   230, 275,               0, 2,   50*60 ) )
    Bins.append( Bin("PCF_L4", "Polar cap F-region",              14, 10,   70,  90,   275, 320,               0, 2,   50*60 ) )
    Bins.append( Bin("PCF_L5", "Polar cap F-region",              14, 10,   70,  90,   320, 365,               0, 2,   50*60 ) )
    Bins.append( Bin("PCF_L6", "Polar cap F-region",              14, 10,   70,  90,   365, 410,               0, 2,   50*60 ) )
    Bins.append( Bin("PCF_L7", "Polar cap F-region",              14, 10,   70,  90,   410, 455,               0, 2,   50*60 ) )
    Bins.append( Bin("PCF_L8", "Polar cap F-region",              14, 10,   70,  90,   455, 500,               0, 2,   50*60 ) )
    Bins.append( Bin("PCF_M1", "Polar cap F-region",              14, 10,   70,  90,   140, 230,               2, 4,   30*60 ) )
    Bins.append( Bin("PCF_M2", "Polar cap F-region",              14, 10,   70,  90,   230, 320,               2, 4,   30*60 ) )
    Bins.append( Bin("PCF_M3", "Polar cap F-region",              14, 10,   70,  90,   320, 410,               2, 4,   30*60 ) )
    Bins.append( Bin("PCF_M4", "Polar cap F-region",              14, 10,   70,  90,   410, 500,               2, 4,   30*60 ) )
    Bins.append( Bin("PCF_H1", "Polar cap F-region",              14, 10,   70,  90,   140, 260,               4, 9,   20*60 ) )
    Bins.append( Bin("PCF_H2", "Polar cap F-region",              14, 10,   70,  90,   260, 380,               4, 9,   20*60 ) )
    Bins.append( Bin("PCF_H3", "Polar cap F-region",              14, 10,   70,  90,   380, 500,               4, 9,   20*60 ) )
    binGroupNames = list()
    for B in Bins:
        aGroupName = B.ID[ : B.ID.find("_") ]
        if aGroupName not in binGroupNames: binGroupNames.append( aGroupName )
    BinGroups_Dropdown.options = binGroupNames
    
InitializeBins()


def createBinsForTheWholeEarth():
    n = 1
    for Kp_min in [0, 2, 4]:
        if Kp_min == 0:
            Kp_max = 2
        elif Kp_min == 2:
            Kp_max = 4
        elif Kp_min == 4:
            Kp_max = 9
        ####matc
        for MLT_min in range(0, 24, 4):
            for MagLat_min in range(-180, 180, 20):
                for Alt_min in range(115, 250, 25):
                    n = n + 1
                    Bins.append( Bin("E"+str(n), "",            MLT_min, MLT_min+4,   MagLat_min, MagLat_min+15,   Alt_min, Alt_min+25,               Kp_min, Kp_max,   20*60 ) )                    
    print ( len(Bins) + " Bins covering the whole Earth.")    
#createBinsForTheWholeEarth()

# cheks if certain MLT lies in a certain range. Created in order to take account ranges like 22-2
def is_MLT_inside_range( MLT, MLT_min, MLT_max ):
    if MLT_min <= MLT_max: # example: from 13 to 18 hour
        return (MLT > MLT_min  and  MLT <= MLT_max)
    else: # example: from 22 to 3 hour
        return (MLT > MLT_min  or   MLT <= MLT_max)

# returns: the bin object which matches the arguments
def GetMatchedBin( MLT, MagLat, Altitude, Kp ):
    MatchedBin = None
    for B in Bins:
        if is_MLT_inside_range(MLT, B.MLT_min, B.MLT_max):
            if         MagLat   > B.MagLat_min    and  MagLat   <= B.MagLat_max:
                if     Altitude > B.Altitude_min  and  Altitude <= B.Altitude_max:
                    Kp_min_to_check = B.Kp_min
                    if Kp_min_to_check == 0: Kp_min_to_check = -1
                    if Kp       > Kp_min_to_check and  Kp       <= B.Kp_max:
                        MatchedBin = B
                        break
    return MatchedBin


# returns: the bin object which matches the arguments
def getBinByItsProperties( MLT_min, MLT_max, MagLat_min, MagLat_max, Altitude_min, Altitude_max, Kp_min, Kp_max ):
    CorrectBin = None
    for B in Bins:
        if             MLT_min      == B.MLT_min       and  MLT_max      == B.MLT_max:
            if         MagLat_min   == B.MagLat_min    and  MagLat_max   == B.MagLat_max:
                if     Altitude_min == B.Altitude_min  and  Altitude_max == B.Altitude_max:
                    if Kp_min       == B.Kp_min        and  Kp_max       == B.Kp_max:
                        CorrectBin = B
                        break
    return CorrectBin

# returns: the bin object which matches the arguments
def getBinByItsID( aBinID ):
    CorrectBin = None
    for B in Bins:
        if  B.ID == aBinID:
            CorrectBin = B
            break
    return CorrectBin


        
# Save the results in a text file        
def SaveResults( ResultsFilename ):
    global CALCULATIONS_Title, CALCULATIONS_Description, CALCULATIONS_RegionName, CALCULATIONS_OrbitFilename, CALCULATIONS_DataPath, CALCULATIONS_ExecutionDuration
    global all_JH_values, all_MagLat_values, all_MLT_values, all_Altitude_values
    nowstr = datetime.now().strftime("%d-%m-%Y %H:%M:%S")    
    F = open(ResultsFilename, 'w')
    F.write( "# -- JOULE HEATING per BIN RESULTS -- " + "\n"  )
    F.write( "# Date of execution: " + nowstr + "\n" )
    F.write( "# Title: " + CALCULATIONS_Title + "\n" )
    F.write( "# Region: " + CALCULATIONS_RegionName + "\n" )
    F.write( "# Orbit Filename: " + CALCULATIONS_OrbitFilename + "\n" )
    F.write( "# Description: " + CALCULATIONS_Description + "\n")
    F.write( "# DataPath: " + CALCULATIONS_DataPath + "\n")
    F.write( "# Duration of execution: " + ConvertLeadingZerosToSpaces("{0:.0f}".format(CALCULATIONS_ExecutionDuration)) + " seconds  or  " + ConvertLeadingZerosToSpaces("{0:.2f}".format(CALCULATIONS_ExecutionDuration/60))  + " minutes" + "\n" )
    F.write( "# " + "\n")    
    for B in Bins:
        F.write( B.getInfo() + "\n" )
    ##
    F.write( "\nAll JH values: " ) 
    for i in range(0, len(all_JH_values) ):
        F.write( str( all_JH_values[i]) )
        if i < len(all_JH_values)-1: F.write( ',' )
    F.write( "\nAll MagLat values: " ) 
    for i in range(0, len(all_MagLat_values) ):
        F.write( "{:.4g}".format( all_MagLat_values[i]) )
        if i < len(all_MagLat_values)-1: F.write( ',' )   
    F.write( "\nAll MLT values: " ) 
    for i in range(0, len(all_MLT_values) ):
        F.write( "{:.4g}".format( all_MLT_values[i]) )
        if i < len(all_MLT_values)-1: F.write( ',' ) 
    F.write( "\nAll Altitude values: " ) 
    for i in range(0, len(all_Altitude_values) ):
        F.write( "{:.4g}".format( all_Altitude_values[i]) )
        if i < len(all_Altitude_values)-1: F.write( ',' )             
    ## write all data separated at bins
    F.write("\n")
    for B in Bins:
        F.write("\n")
        F.write( "BIN " + B.ID + ": MagLat values = " )
        for i in range(0, len(B.MagLat_values) ):
            F.write(  "{:.5g}".format(B.MagLat_values[i])  )
            if i < len(B.MagLat_values)-1: F.write( ',' )
        F.write("\n")
        F.write( "BIN " + B.ID + ": MLT values = " )
        for i in range(0, len(B.MLT_values) ):
            F.write(  "{:.5g}".format(B.MLT_values[i])  )
            if i < len(B.MLT_values)-1: F.write( ',' )
        F.write("\n")
        F.write( "BIN " + B.ID + ": Altitude values = " )
        for i in range(0, len(B.Altitude_values) ):
            F.write(  "{:.5g}".format(B.Altitude_values[i])  )
            if i < len(B.Altitude_values)-1: F.write( ',' )
    ##
    F.close()
    
    
    
    
def LoadResults( filename ):
    global CALCULATIONS_Title, CALCULATIONS_Description, CALCULATIONS_RegionName, CALCULATIONS_OrbitFilename, CALCULATIONS_DataPath, CALCULATIONS_ExecutionDuration
    global all_JH_values, all_MagLat_values, all_MLT_values, all_Altitude_values
    if len(filename) > 0:
        CALCULATIONS_ResultsFilename = ResultsFilename = filename
        with open(CALCULATIONS_ResultsFilename, 'r') as F:
            for line in F:
                if line[0:1] == '#': # this line contains a comment, print it as it is.
                    print ( line[1:len(line)-1] )
                    if line.startswith("# Title:"): CALCULATIONS_Title = line[8:].strip()
                    if line.startswith("# Description:"): CALCULATIONS_Description = line[14:].strip()
                    if line.startswith("# Region:"): CALCULATIONS_RegionName = line[9:].strip()
                    if line.startswith("# Orbit Filename:"): CALCULATIONS_OrbitFilename = line[17:].strip()
                    if line.startswith("# DataPath"): CALCULATIONS_DataPath = line[10:].strip()
                elif line.startswith( "All JH values" ) :
                    all_JH_values = line[line.find(":")+1:].split(',')
                    all_JH_values = [float(i) for i in all_JH_values]
                elif line.startswith( "All MagLat values" ) :
                    all_MagLat_values = line[line.find(":")+1:].split(',')
                    all_MagLat_values = [float(i) for i in all_MagLat_values]
                elif line.startswith( "All MLT values" ) :
                    all_MLT_values = line[line.find(":")+1:].split(',')
                    all_MLT_values = [float(i) for i in all_MLT_values]
                elif line.startswith( "All Altitude values" ) :
                    all_Altitude_values = line[line.find(":")+1:].split(',')
                    all_Altitude_values = [float(i) for i in all_Altitude_values]                    
                elif line.startswith( "BIN" ):
                    head_of_line = line[0:20]
                    bin_id = head_of_line[ 4: head_of_line.find(':') ]
                    B = getBinByItsID( bin_id )
                    data_str = line[line.find("=")+1:-1].strip()
                    if len( data_str ) > 0:
                        values = data_str.split(',')
                        values = [float(i) for i in values]
                    else:
                        values = list()
                    if   "MagLat"   in head_of_line: 
                        B.MagLat_values   = values
                    elif "MLT"      in head_of_line: 
                        B.MLT_values      = values
                    elif "Altitude" in head_of_line: 
                        B.Altitude_values = values    
                else: # this line contains bin info, print it and store them in the correct bin.
                    s = line[:220]
                    if s[-1] == '\n': s = s[:-1]
                    print ( s )
                    aBinID = line[:line.find(":")].strip()
                    ##
                    str_JH = line[line.find("JH_values=")+10:-1]
                    if len( str_JH.strip() ) > 0:
                        aBinJH_values = str_JH.split(',')
                        aBinJH_values = [float(i) for i in aBinJH_values]
                        for B in Bins:
                            if B.ID == aBinID:
                                B.JH_values = aBinJH_values
                                break
        F.close()
        CalculateJouleHeatingPerBin()
    
    
    
    
    
    
@cuda.jit
def Match(MAGLATs, MLTs, ALTs, KPs, JHs,   MagLat_min, MagLat_max, MLT_min,MLT_max, Altitude_min,Altitude_max, Kp_min,Kp_max,   out):
    length_time =  60
    length_lev  =  57
    length_lat  =  72
    length_lon  = 144
    num_of_blocks  = cuda.gridDim.x   # number of blocks in the grid
    num_of_threads = cuda.blockDim.x  # number of threads per block
    BlockIDX  = cuda.blockIdx.x  # this is the unique block ID within the 1D grid
    ThreadIDX = cuda.threadIdx.x # this is the unique thread ID within a 1D block
    #start = tx + ty * block_size
    #stride = block_size * grid_size
    BlockStep  = int(length_lat/num_of_blocks)+1
    ThreadStep = int(length_lon/num_of_threads)+1
    for idx_lat in range( BlockIDX*BlockStep,  BlockIDX*BlockStep + BlockStep):
        if idx_lat >= length_lat: continue
        for idx_lon in range( ThreadIDX*ThreadStep,  ThreadIDX*ThreadStep + ThreadStep):
            if idx_lon >= length_lon: continue
            for idx_lev in range(0, length_lev):
                if idx_lev >= length_lev: continue
                for idx_time in range(0, length_time):
                    if idx_time >= length_time: continue
                    if MAGLATs[idx_time, idx_lev, idx_lat, idx_lon]      >= MagLat_min       and MAGLATs[idx_time, idx_lev, idx_lat, idx_lon] <= MagLat_max:
                        if ALTs[idx_time, idx_lev, idx_lat, idx_lon] >= Altitude_min     and ALTs[idx_time, idx_lev, idx_lat, idx_lon]    <= Altitude_max:
                            if KPs[idx_time] >= Kp_min and KPs[idx_time] <= Kp_max:
                                is_MLT_in_range = False
                                if MLT_min <= MLT_max: # example: from 13 to 18 hour
                                    if MLTs[idx_time, idx_lev, idx_lat, idx_lon] >= MLT_min and MLTs[idx_time, idx_lev, idx_lat, idx_lon] <= MLT_max: is_MLT_in_range = True
                                else: # example: from 22 to 3 hour
                                    if MLTs[idx_time, idx_lev, idx_lat, idx_lon] > MLT_min or MLTs[idx_time, idx_lev, idx_lat, idx_lon] <= MLT_max: is_MLT_in_range = True
                                if is_MLT_in_range: out[idx_time, idx_lev, idx_lat, idx_lon] = JHs[idx_time, idx_lev, idx_lat, idx_lon]

def AssignJouleHeatingValuesToBins_CUDA( DataFilesPath ): # MagLat MagLoacalTime Kp Alt
    startSecs = time.time()        
    MagLat_min =  1000
    MagLat_max = -1000
    MLT_min    =  1000
    MLT_max    = -1000
    Altitude_min    =  1000
    Altitude_max    = -1000
    Kp_min     =  1000
    Kp_max     = -1000
    for B in Bins:
        B.reset()
        if B.MagLat_min < MagLat_min: MagLat_min = B.MagLat_min 
        if B.MagLat_max > MagLat_max: MagLat_max = B.MagLat_max
        if B.MLT_min < MLT_min: MLT_min = B.MLT_min 
        if B.MLT_max > MLT_max: MLT_max = B.MLT_max
        if B.Altitude_min < Altitude_min: Altitude_min = B.Altitude_min 
        if B.Altitude_max > Altitude_max: Altitude_max = B.Altitude_max
        if B.Kp_min < Kp_min: Kp_min = B.Kp_min 
        if B.Kp_max > Kp_max: Kp_max = B.Kp_max            
        
    Matches = 0
    
    AllDataFiles = sorted( glob.glob( DataFilesPath + "*/*.nc", recursive=True ) )
    for currentDataFile in AllDataFiles:
        if "Vars" in currentDataFile: continue
        print( "Reading", currentDataFile )
        try:
            CDFroot = Dataset( currentDataFile, 'r' )
        except:
            print ( "WRONG FORMAT:", currentDataFile )
            continue
        try:
            foo = CDFroot.variables['Ohmic'][0,0,0,0]                        
        except:
            print ( "WRONG FORMAT inside:", currentDataFile )
            continue
        #FileStartTimeStamp = calendar.timegm( datetime.strptime( CDFroot.variables['time'].units[14:],  "%Y-%m-%d %H:%M:%S" ).utctimetuple() ) # ex: "minutes since 2015-1-1 0:0:0"
        FileStartTimeStamp = calendar.timegm( datetime.strptime( "minutes since 2015-1-1 0:0:0"[14:],  "%Y-%m-%d %H:%M:%S" ).utctimetuple() )
        length_time = CDFroot.variables['Ohmic'].shape[0]
        length_lev  = CDFroot.variables['Ohmic'].shape[1]
        length_lat  = CDFroot.variables['Ohmic'].shape[2]
        length_lon  = CDFroot.variables['Ohmic'].shape[3]
        
        # Load or calculate all basic values from the netcdf file
        print ("Loading file into memory")
        #TIMEs   = CDFroot.variables['time'][:] # minutes since the start time
        #LATs    = CDFroot.variables['lat'] # geographic latitude (-south, +north)
        #LONs    = CDFroot.variables['lon'] # geographic longitude (-west, +east)
        MAGLATs = CDFroot.variables['mlat_qdf'][:, :, :, :] 
        MLTs    = CDFroot.variables['mlt_qdf'][:, :, :, :]         
        ALTs    = CDFroot.variables['zgmid'][:, :, :, :] / 100000 # it is stored in cm inside the file
        KPs     = CDFroot.variables['kp'][:]
        JHs     = CDFroot.variables['Ohmic'][:, :, :, :]
        
        print( "Invoking CUDA" )
        MAGLATs = np.array( MAGLATs, dtype="float32" )
        MLTs = np.array( MLTs, dtype="float32" )        
        ALTs = np.array( ALTs, dtype="float32" )
        KPs = np.array( KPs, dtype="float32" )
        JHs = np.array( JHs, dtype="float32" )
        out = np.zeros(  [length_time, length_lev, length_lat, length_lon]  )
        blocks_per_grid = (16)
        threads_per_block = (32)
        Match[blocks_per_grid, threads_per_block](MAGLATs, MLTs, ALTs, KPs, JHs,   MagLat_min, MagLat_max, MLT_min,MLT_max, Altitude_min,Altitude_max, Kp_min,Kp_max,   out)
        #device_out.copy_to_host()
        print( "Positions in Bins:", np.count_nonzero(out), "/" , out.size)
        
        for idx_lat in range(0, length_lat):
            for idx_lon in range(0, length_lon):
                for idx_lev in range(0, length_lev):
                    for idx_time in range(0, length_time):
                        if out[idx_time, idx_lev, idx_lat ,idx_lon] != 0:
                            current_MLT      = MLTs[idx_time, idx_lev, idx_lat ,idx_lon]
                            current_MagLat   = MAGLATs[idx_time, idx_lev, idx_lat ,idx_lon]
                            current_Altitude = ALTs[idx_time, idx_lev, idx_lat ,idx_lon]
                            current_Kp       = KPs[idx_time]
                            mathedBin = GetMatchedBin( current_MLT, current_MagLat, current_Altitude, current_Kp )
                            if mathedBin is not None:
                                current_JH = JHs[idx_time, idx_lev, idx_lat ,idx_lon] #CDFroot.variables['Joule Heating'][idx_time, idx_lev, idx_lat, idx_lon]
                                mathedBin.JH_values.append( current_JH )
                                mathedBin.MagLat_values.append( current_MagLat )
                                mathedBin.MLT_values.append( current_MLT )
                                mathedBin.Altitude_values.append( current_Altitude )
                                all_JH_values.append( current_JH )
                                all_MagLat_values.append( current_MagLat )
                                all_MLT_values.append( current_MLT )
                                all_Altitude_values.append( current_Altitude )
                                Matches += 1
        
        CDFroot.close()
        print( Matches, " matches so far." )
    finishSecs = time.time()
    print( finishSecs-startSecs, " sec")    
    print( Matches, "points have been assigned into bins" )

            
#######################            #######################            #######################            
                    #######################            #######################            
def AssignJouleHeatingValuesToBins( DataFilesPath ): # MagLat MagLoacalTime Kp Alt
    startSecs = time.time()
    MagLat_min =  1000
    MagLat_max = -1000
    MLT_min    =  1000
    MLT_max    = -1000
    Altitude_min    =  1000
    Altitude_max    = -1000
    Kp_min     =  1000
    Kp_max     = -1000
    for B in Bins:
        B.reset()
        if B.MagLat_min < MagLat_min: MagLat_min = B.MagLat_min 
        if B.MagLat_max > MagLat_max: MagLat_max = B.MagLat_max
        if B.MLT_min < MLT_min: MLT_min = B.MLT_min 
        if B.MLT_max > MLT_max: MLT_max = B.MLT_max
        if B.Altitude_min < Altitude_min: Altitude_min = B.Altitude_min 
        if B.Altitude_max > Altitude_max: Altitude_max = B.Altitude_max
        if B.Kp_min < Kp_min: Kp_min = B.Kp_min 
        if B.Kp_max > Kp_max: Kp_max = B.Kp_max            
        
    #print( MagLat_min,    MagLat_max,    MLT_min,    MLT_max , Altitude_min,    Altitude_max,     Kp_min,     Kp_max)
    Matches = 0
    
    AllDataFiles = sorted( glob.glob( DataFilesPath + "*.nc" ) )
    for currentDataFile in AllDataFiles:
        print( "Reading", currentDataFile )
        try:
            CDFroot = Dataset( currentDataFile, 'r' )
        except:
            print ( "WRONG FORMAT:", currentDataFile )
            continue
        try:
            foo = CDFroot.variables['Ohmic'][0,0,0,0]                        
        except:
            print ( "WRONG FORMAT inside:", currentDataFile )
            continue
        FileStartTimeStamp = calendar.timegm( datetime.strptime( CDFroot.variables['time'].units[14:],  "%Y-%m-%d %H:%M:%S" ).utctimetuple() ) # ex: "minutes since 2015-1-1 0:0:0"
        #FileStartTimeStamp = calendar.timegm( datetime.strptime( "minutes since 2015-1-1 0:0:0"[14:],  "%Y-%m-%d %H:%M:%S" ).utctimetuple() ) 
        length_time = CDFroot.variables['Ohmic'].shape[0]
        length_lev  = CDFroot.variables['Ohmic'].shape[1]
        length_lat  = CDFroot.variables['Ohmic'].shape[2]
        length_lon  = CDFroot.variables['Ohmic'].shape[3]
        
        # Load or calculate all basic values from the netcdf file
        print ("Loading file into memory")
        #TIMEs   = CDFroot.variables['time'][:] # minutes since the start time
        #LATs    = CDFroot.variables['lat'] # geographic latitude (-south, +north)
        #LONs    = CDFroot.variables['lon'] # geographic longitude (-west, +east)
        ALTs    = CDFroot.variables['ZGMID'][:, :, :, :] / 100000 # it is stored in cm inside the file
        JHs     = CDFroot.variables['Ohmic'][:, :, :, :]
        KPs     = CDFroot.variables['Kp'][:]
        MAGLATs = CDFroot.variables['mlat_qdf'][:, :, :, :] 
        MLTs    = CDFroot.variables['mlt_qdf'][:, :, :, :] 
        step = 1
        for idx_lat in range(0, length_lat, step):
            if idx_lat%5==0: print("Calculating Lat",  idx_lat)
            for idx_lon in range(0, length_lon, step):
                for idx_lev in range(0, length_lev, step):
                    for idx_time in range(0, length_time, step):
                        in_Altitude_range = in_MagLat_range = in_MLT_range = in_Kp_range = False
                        
                        current_Altitude = ALTs[idx_time, idx_lev, idx_lat, idx_lon]
                        if current_Altitude >= Altitude_min and current_Altitude <= Altitude_max:
                                in_Altitude_range = True
                        
                        if in_Altitude_range:
                            #current_Latitude  = LATs[ idx_lat ]    current_Longitude = LONs[ idx_lon ]     geodetic_Latitude = Conversions.geo_lat2geod_lat( current_Latitude )    TimeObj = datetime.fromtimestamp( FileStartTimeStamp + 60*TIMEs[idx_time], tz=timezone.utc )       current_MagLat, current_MagLon, current_MLT = Conversions.getMagneticProperties( TimeObj, geodetic_Latitude, current_Longitude, current_Altitude )
                            current_MagLat = MAGLATs[ idx_time, idx_lev, idx_lat, idx_lon ]
                            if current_MagLat >= MagLat_min and current_MagLat <= MagLat_max:
                                in_MagLat_range = True
                                
                        if in_MagLat_range:
                            current_MLT = MLTs[ idx_time, idx_lev, idx_lat, idx_lon ]
                            if in_MagLat_range:
                                in_MLT_range = is_MLT_inside_range( current_MLT, MLT_min, MLT_max )
                        
                        if in_MLT_range:
                            current_Kp = KPs[idx_time]
                            if current_Kp >= Kp_min and current_Kp <= Kp_max:
                                in_Kp_range = True                                    
                        ##
                        if in_MagLat_range and in_MLT_range and in_Altitude_range and in_Kp_range:
                            mathedBin = GetMatchedBin( current_MLT, current_MagLat, current_Altitude, current_Kp )
                            if mathedBin is not None:
                                current_JH = JHs[idx_time, idx_lev, idx_lat ,idx_lon] #CDFroot.variables['Joule Heating'][idx_time, idx_lev, idx_lat, idx_lon]
                                mathedBin.JH_values.append( current_JH )
                                mathedBin.MagLat_values.append( current_MagLat )
                                mathedBin.MLT_values.append( current_MLT )
                                mathedBin.Altitude_values.append( current_Altitude )
                                all_JH_values.append( current_JH )
                                all_MagLat_values.append( current_MagLat )
                                all_MLT_values.append( current_MLT )
                                all_Altitude_values.append( current_Altitude )
                                Matches += 1
                            else:
                                print( "PARADOX at:", current_MLT, current_MagLat, current_Altitude, current_Kp, " :: ", idx_time, idx_lev, idx_lat, idx_lon )
                #break
            #break
        CDFroot.close()
        #if "_02." in currentDataFile: break # >>>>>> FOR TEST so that it reads only one file
    finishSecs = time.time()
    print( finishSecs-startSecs, " sec")    
    print( Matches, "points have been assigned into bins" )

    
    
    
    
'''
TIEGCM_filesPath: the folder which has all TIEGCM netcdf files describing Earth's enviroment during the orbit's duration
OrbitFilesPath: the folder which has all netCDF files which contain all the positions of the satellite 
'''    
def AssignJouleHeatingValuesToBins_AlongOrbit( TIEGCM_filesPath, Orbit_filesPath ): # MagLat MagLoacalTime Kp Alt 
    # initialize
    MagLat_min =  1000
    MagLat_max = -1000
    MLT_min    =  1000
    MLT_max    = -1000
    Altitude_min    =  1000
    Altitude_max    = -1000
    Kp_min     =  1000
    Kp_max     = -1000
    for B in Bins:
        B.reset()
        if B.MagLat_min < MagLat_min: MagLat_min = B.MagLat_min 
        if B.MagLat_max > MagLat_max: MagLat_max = B.MagLat_max
        if B.MLT_min < MLT_min: MLT_min = B.MLT_min 
        if B.MLT_max > MLT_max: MLT_max = B.MLT_max
        if B.Altitude_min < Altitude_min: Altitude_min = B.Altitude_min 
        if B.Altitude_max > Altitude_max: Altitude_max = B.Altitude_max
        if B.Kp_min < Kp_min: Kp_min = B.Kp_min 
        if B.Kp_max > Kp_max: Kp_max = B.Kp_max            
        
    Matches = 0

    # read orbit file
    AllOrbitFiles = sorted( glob.glob( Orbit_filesPath + "*.nc" ) )
    for currentOrbitFile in AllDataFiles:
        print( "Reading", currentOrbitFile )
        try:
            CDFroot = Dataset( currentOrbitFile, 'r' )
        except:
            print ( "WRONG FORMAT:", currentDataFile )
            continue
        # Load data from the netCDF file
        DATA_Ohmic = CDFroot.variables['Ohmic'][:]
        num_of_positions =  len(DATA_Ohmic)
        # read the satellite positions and try to fill the bins
        for idx in range(0, num_of_positions): # for each satellite position
            if idx % 200000 == 0: print ("Checking sat position No", line_count, "of", num_of_positions)
            current_GeodLat = float( row[Lat_col] )
            current_GeogLat = float( row[Lat_col] ) # TODO: read correct column from orbit file
            current_Lon = float( row[Lon_col] )
            current_Altitude = float( row[Altitude_col] )
            in_Altitude_range = in_MagLat_range = in_MLT_range = in_Kp_range = False
                      
            # check if this position lies inside some bin
            current_Altitude = float( row[Altitude_col] )
            if current_Altitude >= Altitude_min and current_Altitude <= Altitude_max:
                in_Altitude_range = True
            #
            if in_Altitude_range:
                current_MagLat = float( row[MagLat_col] )
                if current_MagLat >= MagLat_min and current_MagLat <= MagLat_max:
                    in_MagLat_range = True
            #
            if in_MagLat_range:
                current_MLT = float( row[MLT_col] )
                in_MLT_range = is_MLT_inside_range( current_MLT, MLT_min, MLT_max )

            # The position is probably inside a bin (only kp remains to be checked). 
            # Open the corresponding TIEGCM file to read the kp and if position is in bin then calculate JH
            if in_MLT_range:
                current_time = parseDaedalusDate( row[Time_col] )
                if current_time == None:
                    print( "ERROR during coverage calculation while reading", OrbitFilename, ": Wrong time format:", row[Time_col], "at line", line_count )
                    return # <<<<
                # TODO remove after testing:
                if current_time.year > 2024:  current_time = current_time - relativedelta(years=13)
                # open the correct TIEGCM file according to time
                current_timestep_number = (calendar.timegm(current_time.utctimetuple()) - TIEGCM_StartTimeStamp) / TIEGCM_TimeStep_sec
                newfilenumber = int(( current_timestep_number ) / TIEGCM_NumOfTimeSteps) + 1
                
                #print( "ZAZA", "orbit t=", current_time , "timestep_num=", current_timestep_number, "  filenum=", newfilenumber )
                if newfilenumber<=1 or newfilenumber > 73: continue # TODO del this line
                    
                if currentfilenumber < 0   or   currentfilenumber != newfilenumber:
                    if currentfilenumber >= 0: CDFroot.close()
                    TIEGCMfilename = TIEGCM_filesPath + TIEGCMfilenamePrefix + "{:03.0f}".format(newfilenumber) + TIEGCMfilenamePostfix + ".nc"
                    currentfilenumber = newfilenumber
                    print(  "Opening TIEGCMfile:", TIEGCMfilename)
                    try:
                        CDFroot = Dataset( TIEGCMfilename, 'r' )
                    except:
                        print ( "FILE NOT FOUND OR WRONG FORMAT:", TIEGCMfilename )
                        continue
                # calculate the time-step inside the TIEGCM which corresponds to the satellite time 
                time_idx = int(  current_timestep_number - (newfilenumber-1)*TIEGCM_NumOfTimeSteps  )
                # read Kp from the tiegcm file
                
                current_Kp = CDFroot.variables['Kp'][time_idx]
                if current_Kp >= Kp_min and current_Kp <= Kp_max:
                    in_Kp_range = True 
                # if the satellite position matches a bin then mark it as a hit and remember the JH values 
                if in_MagLat_range and in_MLT_range and in_Altitude_range and in_Kp_range:
                    mathedBin = GetMatchedBin( current_MLT, current_MagLat, current_Altitude, current_Kp )
                    if mathedBin is not None:
                        # for this position locate the neigbor latitudes at the TIEGCM file. 
                        lat1_idx, lat2_idx, lat1_val, lat2_val = findNeighborValues( TIEGCM_Lats, current_GeogLat )
                        # for this position locate the neigbor longitudes at the TIEGCM file.
                        lon1_idx, lon2_idx, lon1_val, lon2_val = findNeighborValues( TIEGCM_Lons, current_Lon )
                        # for this position locate the neigbor Altitudes at the TIEGCM file. 
                        lev1_idx, lev2_idx, lev1_val, lev2_val = findNeighborValues( CDFroot.variables['ZGMID'][time_idx, :, lat_idx, lon_idx], current_Altitude )
                        # TODO: TRILINEAR INTERPOLATION
                        current_JH = CDFroot.variables['Ohmic'][time_idx, lev1_idx, lat1_idx, lon1_idx] 
                        # save 
                        mathedBin.JH_values.append( current_JH )
                        mathedBin.MagLat_values.append( current_MagLat )
                        mathedBin.MLT_values.append( current_MLT )
                        mathedBin.Altitude_values.append( current_Altitude )
                        all_JH_values.append( current_JH )
                        all_MagLat_values.append( current_MagLat )
                        all_MLT_values.append( current_MLT )
                        all_Altitude_values.append( current_Altitude )
                        Matches += 1
                    else:
                        print( "PARADOX at:", current_MLT, current_MagLat, current_Altitude, current_Kp, " :: ", time_idx, lev_idx, lat_idx, lon_idx )
                        
    # clean up
    print( Matches, "satellite positions where matched inside bins." )
    try:
        CDFroot.close()
    except:
        print (".")
    
    
    
'''
TIEGCM_filesPath: the folder which has all TIEGCM netcdf files describing Earth's enviroment during the orbit's duration
OrbitFilename: csv file containing all the positions of the satellite 
'''    
def AssignJouleHeatingValuesToBins_AlongOrbit_forCSVorbit( TIEGCM_filesPath, OrbitFilename ): # MagLat MagLoacalTime Kp Alt 
    # initialize
    MagLat_min =  1000
    MagLat_max = -1000
    MLT_min    =  1000
    MLT_max    = -1000
    Altitude_min    =  1000
    Altitude_max    = -1000
    Kp_min     =  1000
    Kp_max     = -1000
    for B in Bins:
        B.reset()
        if B.MagLat_min < MagLat_min: MagLat_min = B.MagLat_min 
        if B.MagLat_max > MagLat_max: MagLat_max = B.MagLat_max
        if B.MLT_min < MLT_min: MLT_min = B.MLT_min 
        if B.MLT_max > MLT_max: MLT_max = B.MLT_max
        if B.Altitude_min < Altitude_min: Altitude_min = B.Altitude_min 
        if B.Altitude_max > Altitude_max: Altitude_max = B.Altitude_max
        if B.Kp_min < Kp_min: Kp_min = B.Kp_min 
        if B.Kp_max > Kp_max: Kp_max = B.Kp_max            
        
    # information about the TIEGCM files
    TIEGCMfilenamePrefix  = "tiegcm2.0_res2.5_3years_sech_" 
    TIEGCMfilenamePostfix = "_JH_QD_AllVars"
    CDFroot = Dataset( TIEGCM_filesPath + TIEGCMfilenamePrefix + "002" + TIEGCMfilenamePostfix + ".nc", 'r' )  # open a tiegcm file
    TIEGCM_StartTimeStamp  = calendar.timegm( datetime.strptime( CDFroot.variables['time'].units[14:],  "%Y-%m-%d %H:%M:%S" ).utctimetuple() ) # ex: "minutes since 2015-1-1 0:0:0"
    #TIEGCM_StartTimeStamp  = calendar.timegm( datetime.strptime( "minutes since 2015-1-1 0:0:0"[14:],  "%Y-%m-%d %H:%M:%S" ).utctimetuple() ) 
    TIEGCM_TimeStep_sec    = (CDFroot.variables['time'][1] - CDFroot.variables['time'][0]) * 60 # every how many seconds a measurement is stored in the file
    TIEGCM_NumOfTimeSteps  = len( CDFroot.variables['time'][:] ) # the number of timesteps stored in the file
    TIEGCM_Lats = CDFroot.variables['lat'][:]
    TIEGCM_Lons = CDFroot.variables['lon'][:]
    CDFroot.close()
    # miscellaneous
    currentfilenumber = -1
    Matches = 0
    time_idx = lat_idx = lon_idx = lev_idx = -1
    print( "TIEGCM UNIVERSE:")
    print( "    Start Time =", "(UTC:"+str(TIEGCM_StartTimeStamp)+")", datetime.fromtimestamp(TIEGCM_StartTimeStamp) )
    print( "    Time-step  =", str(TIEGCM_TimeStep_sec)+"sec" + " #steps/file =", TIEGCM_NumOfTimeSteps, " Duration/file =", str(TIEGCM_NumOfTimeSteps*TIEGCM_TimeStep_sec/(60*60))+"hours", "\n" )
    
    # read orbit file
    with open( OrbitFilename ) as CSVfile:        
        CSVreader = csv.reader( CSVfile )
        # locate the column numnbers of interest inside the csv file
        CSVheader = next( CSVreader )
        Time_col     = CSVheader.index( "Epoch(UTCG)" ) #CSVheader.index( "Daedalus.EpochText" )
        Lat_col      = CSVheader.index( "Lat_GEOD(deg)" ) #CSVheader.index( "Daedalus.Latitude" )
        Lon_col      = CSVheader.index( "Lon_GEOD(deg)" ) #CSVheader.index( "Daedalus.Longitude" )
        Altitude_col = CSVheader.index( "Height_WGS84 (km)" ) #CSVheader.index( "Daedalus.Height" )
        MagLat_col   = CSVheader.index( "Daedalus.Magnetic Latitude" )
        MLT_col      = CSVheader.index( "Daedalus.MLT" )
        # read the satellite positions and try to fill the bins
        line_count = 0
        for row in CSVreader: # for each satellite position
            line_count += 1
            if line_count % 200000 == 0: print ("Checking sat position No", line_count, "of", row[Time_col])
            current_GeodLat = float( row[Lat_col] )
            current_GeogLat = float( row[Lat_col] ) # TODO: read correct column from orbit file
            current_Lon = float( row[Lon_col] )
            current_Altitude = float( row[Altitude_col] )
            in_Altitude_range = in_MagLat_range = in_MLT_range = in_Kp_range = False
                      
            # check if this position lies inside some bin
            current_Altitude = float( row[Altitude_col] )
            if current_Altitude >= Altitude_min and current_Altitude <= Altitude_max:
                in_Altitude_range = True
            #
            if in_Altitude_range:
                current_MagLat = float( row[MagLat_col] )
                if current_MagLat >= MagLat_min and current_MagLat <= MagLat_max:
                    in_MagLat_range = True
            #
            if in_MagLat_range:
                current_MLT = float( row[MLT_col] )
                in_MLT_range = is_MLT_inside_range( current_MLT, MLT_min, MLT_max )

            # The position is probably inside a bin (only kp remains to be checked). 
            # Open the corresponding TIEGCM file to read the kp and if position is in bin then calculate JH
            if in_MLT_range:
                current_time = parseDaedalusDate( row[Time_col] )
                if current_time == None:
                    print( "ERROR during coverage calculation while reading", OrbitFilename, ": Wrong time format:", row[Time_col], "at line", line_count )
                    return # <<<<
                # TODO remove after testing:
                if current_time.year > 2024:  current_time = current_time - relativedelta(years=13)
                # open the correct TIEGCM file according to time
                current_timestep_number = (calendar.timegm(current_time.utctimetuple()) - TIEGCM_StartTimeStamp) / TIEGCM_TimeStep_sec
                newfilenumber = int(( current_timestep_number ) / TIEGCM_NumOfTimeSteps) + 1
                
                #print( "ZAZA", "orbit t=", current_time , "timestep_num=", current_timestep_number, "  filenum=", newfilenumber )
                if newfilenumber<=1 or newfilenumber > 73: continue # TODO del this line
                    
                if currentfilenumber < 0   or   currentfilenumber != newfilenumber:
                    if currentfilenumber >= 0: CDFroot.close()
                    TIEGCMfilename = TIEGCM_filesPath + TIEGCMfilenamePrefix + "{:03.0f}".format(newfilenumber) + TIEGCMfilenamePostfix + ".nc"
                    currentfilenumber = newfilenumber
                    print(  "Opening TIEGCMfile:", TIEGCMfilename)
                    try:
                        CDFroot = Dataset( TIEGCMfilename, 'r' )
                    except:
                        print ( "FILE NOT FOUND OR WRONG FORMAT:", TIEGCMfilename )
                        continue
                # calculate the time-step inside the TIEGCM which corresponds to the satellite time 
                time_idx = int(  current_timestep_number - (newfilenumber-1)*TIEGCM_NumOfTimeSteps  )
                # read Kp from the tiegcm file
                
                current_Kp = CDFroot.variables['Kp'][time_idx]
                if current_Kp >= Kp_min and current_Kp <= Kp_max:
                    in_Kp_range = True 
                # if the satellite position matches a bin then mark it as a hit and remember the JH values 
                if in_MagLat_range and in_MLT_range and in_Altitude_range and in_Kp_range:
                    mathedBin = GetMatchedBin( current_MLT, current_MagLat, current_Altitude, current_Kp )
                    if mathedBin is not None:
                        # for this position locate the neigbor latitudes at the TIEGCM file. 
                        lat1_idx, lat2_idx, lat1_val, lat2_val = findNeighborValues( TIEGCM_Lats, current_GeogLat )
                        # for this position locate the neigbor longitudes at the TIEGCM file.
                        lon1_idx, lon2_idx, lon1_val, lon2_val = findNeighborValues( TIEGCM_Lons, current_Lon )
                        # for this position locate the neigbor Altitudes at the TIEGCM file. 
                        lev1_idx, lev2_idx, lev1_val, lev2_val = findNeighborValues( CDFroot.variables['ZGMID'][time_idx, :, lat_idx, lon_idx], current_Altitude )
                        # TODO: TRILINEAR INTERPOLATION
                        current_JH = CDFroot.variables['Ohmic'][time_idx, lev1_idx, lat1_idx, lon1_idx] 
                        # save 
                        mathedBin.JH_values.append( current_JH )
                        mathedBin.MagLat_values.append( current_MagLat )
                        mathedBin.MLT_values.append( current_MLT )
                        mathedBin.Altitude_values.append( current_Altitude )
                        all_JH_values.append( current_JH )
                        all_MagLat_values.append( current_MagLat )
                        all_MLT_values.append( current_MLT )
                        all_Altitude_values.append( current_Altitude )
                        Matches += 1
                    else:
                        print( "PARADOX at:", current_MLT, current_MagLat, current_Altitude, current_Kp, " :: ", time_idx, lev_idx, lat_idx, lon_idx )
                        
    # clean up
    print( Matches, "satellite positions where matched inside bins." )
    try:
        CDFroot.close()
    except:
        print (".")
    

    

# Finds the neighbor values of <aValue> inside the list.    
# aList: a list of floats, with ascending sorting
# aValue: a float value
# RETURNS:
#    the value of the lesser Neighbor, the value of the greater Neighbor, the index of the lesser Neighbor, the index of the greater Neighbor, 
def findNeighborValues( aList, aValue ):
    listlength = len(aList)
    stop_idx = -1
    for i in range( 0, listlength ):
        if aValue < aList[i]:
            stop_idx = i
            break
    if stop_idx == -1: # <aValue> is greater than all the values of the list
        LesserNeighborIdx  = listlength - 1
        GreaterNeighborIdx = 0
    elif stop_idx == 0: # <aValue> is lesser than all the values of the list
        LesserNeighborIdx  = listlength - 1
        GreaterNeighborIdx = 0
    else:
        LesserNeighborIdx  = stop_idx-1
        GreaterNeighborIdx = stop_idx
    #
    return LesserNeighborIdx, GreaterNeighborIdx, aList[LesserNeighborIdx], aList[GreaterNeighborIdx]
        
    
    
def CalculateJouleHeatingPerBin():
    for B in Bins:
        if len(B.JH_values) > 0: 
            for aJHvalue in B.JH_values:
                if B.JH_min > aJHvalue: B.JH_min = aJHvalue
                if B.JH_max < aJHvalue: B.JH_max = aJHvalue
                B.JH_mean += aJHvalue
            B.JH_mean = B.JH_mean / len(B.JH_values)
            # for Variance:
            for aJHvalue in B.JH_values:
                B.JH_variance += abs(aJHvalue - B.JH_mean)**2
            B.JH_variance = B.JH_variance / len(B.JH_values)
            #print( "|||||||||||||| ",  statistics.mean(B.JH_values), B.JH_mean )
            #print( ":::::::::::::: ",  statistics.variance(B.JH_values), B.JH_variance,  B.JH_variance/len(B.JH_values))
            

        
        

#################### EVENT LISTENERS ###########################
def Exec_Btn_Clicked( b ):
    global Bins
    global CALCULATIONS_Title, CALCULATIONS_Description, CALCULATIONS_RegionName, CALCULATIONS_OrbitFilename, CALCULATIONS_DataPath, CALCULATIONS_ExecutionDuration
    CALCULATIONS_Title         = ExecutionTitle_Text.value
    CALCULATIONS_Description   = ExecutionDescr_Text.value
    CALCULATIONS_RegionName    = BinGroups_Dropdown.value
    CALCULATIONS_OrbitFilename = ""
    CALCULATIONS_DataPath      = tiegcmFolder_Dropdown.value
    ResultsFilename = DaedalusGlobals.CoverageResults_Files_Path + BinGroups_Dropdown.value + "." + CALCULATIONS_DataPath[CALCULATIONS_DataPath[:-1].rfind('/')+1:-1] + ".JHperBinResults.txt"
    if path.exists( ResultsFilename ):
        print( "File " + ResultsFilename + " already exists. Cannot continue in order to prevent overwriting useful data." )
    else:
        # remove all other bin-groups so that calculation is faster
        newBins = list()
        for B in Bins:
            if B.ID.startswith( BinGroups_Dropdown.value ): newBins.append( B )
        Bins = newBins
        CALCULATIONS_RegionName = Bins[0].Description + " (" + Bins[0].ID[:3] + ")"
        # do it
        startSecs = time.time()
        print( "Joule-Heating-per-Bin calculation started.", datetime.now() )
        AssignJouleHeatingValuesToBins_CUDA( CALCULATIONS_DataPath )
        CalculateJouleHeatingPerBin()
        finishSecs = time.time()   
        CALCULATIONS_ExecutionDuration = finishSecs-startSecs
        SaveResults( ResultsFilename ) 
        # print info
        print( "Duration", CALCULATIONS_ExecutionDuration, "seconds." )
        print( "Joule-Heating-per-Bin Calculation finshed in " + str(CALCULATIONS_ExecutionDuration) + " seconds." )
        print( "RESULTS (stored in " + ResultsFilename + "):" )
        for B in Bins:
            B.printMe()
        # re-initialize the bins
        InitializeBins()
        # plot
        print( "Please re-run the notebook and load the resutls in order to plot them." )
        #plotAll()
        #plotAll_perKp()

        
def Exec_Btn_alongOrbit_Clicked( b ):
    global Bins
    global CALCULATIONS_Title, CALCULATIONS_Description, CALCULATIONS_RegionName, CALCULATIONS_OrbitFilename, CALCULATIONS_DataPath, CALCULATIONS_ExecutionDuration
    CALCULATIONS_Title         = ExecutionTitle_Text.value
    CALCULATIONS_Description   = ExecutionDescr_Text.value
    CALCULATIONS_RegionName    = BinGroups_Dropdown.value
    CALCULATIONS_OrbitFilename = OrbitFilename_Dropdown.value
    CALCULATIONS_DataPath      = tiegcmFolder_Dropdown.value
    ResultsFilename = DaedalusGlobals.CoverageResults_Files_Path + BinGroups_Dropdown.value + "." + CALCULATIONS_DataPath[CALCULATIONS_DataPath[:-1].rfind('/')+1:-1] + "." + CALCULATIONS_OrbitFilename[CALCULATIONS_OrbitFilename[:-1].rfind('/')+1:-4] + ".JHperBinResults.txt"
    if path.exists( ResultsFilename ):
        print( "File " + ResultsFilename + " already exists. Cannot continue in order to prevent overwriting useful data." )
    else:
        # remove all other bin-groups so that calculation is faster
        newBins = list()
        for B in Bins:
            if B.ID.startswith( BinGroups_Dropdown.value ): newBins.append( B )
        Bins = newBins
        # calculate
        print( "Joule-Heating-per-Bin-Along-Orbit calculation started.", datetime.now() )
        print( "Reading TIEGCM file from", CALCULATIONS_DataPath )
        print( "Reading Orbit file", CALCULATIONS_OrbitFilename )
        print( "Results will be stored in", ResultsFilename, "\n" )
        startSecs = time.time()
        AssignJouleHeatingValuesToBins_AlongOrbit( CALCULATIONS_DataPath, CALCULATIONS_OrbitFilename )
        finishSecs = time.time()   
        CALCULATIONS_ExecutionDuration = finishSecs-startSecs
        SaveResults( ResultsFilename ) 
        # print info
        print( "Duration", CALCULATIONS_ExecutionDuration, "seconds." )
        print( "Joule-Heating-per-Bin Calculation finshed in " + str(CALCULATIONS_ExecutionDuration) + " seconds." )
        print( "RESULTS (stored in " + ResultsFilename + "):" )
        for B in Bins:
            B.printMe()
        # re-initialize the bins
        InitializeBins()




        
def Load_Btn_Clicked( b ):
    global CALCULATIONS_Title, CALCULATIONS_Description, CALCULATIONS_RegionName, CALCULATIONS_OrbitFilename, CALCULATIONS_DataPath, CALCULATIONS_ExecutionDuration
    global all_JH_values, all_MagLat_values, all_MLT_values, all_Altitude_values
    if len(SavedFilenames_Dropdown.value) > 0:
        LoadResults( SavedFilenames_Dropdown.value )
        Plot_JH_Distribution_perBin()
        plotAll()        
        plotAll_perKp()
        #Plot_KpScatter(CALCULATIONS_OrbitFilename, 0, 500)
        
        
        
def CompareResults_Btn_Clicked( b ):
    plotComparisonOfResults()
            
################################################################

def tiegcmFolder_Dropdown_onChange(change):
    if change['type']=='change' and change['name']=='value' and len(change['new'])>0:
        return

        
def SavedFilenames_Dropdown_onChange(change):
    if change['type']=='change' and change['name']=='value' and len(change['new'])>0:
        file_size_in_Gigabytes = os.stat(change['new']).st_size / 1024 / 1024 / 1024
        if file_size_in_Gigabytes > 5:
            Warning_HTML.value= "<b><font color='red'>File size is " + '{0:.1f}'.format(file_size_in_Gigabytes) + " Gigabyte. Ploting will take several minutes.</b>" 
            Warning_HTML.visible=True
        elif file_size_in_Gigabytes > 1:
            Warning_HTML.value= "<b><font color='red'>File size is " + '{0:.1f}'.format(file_size_in_Gigabytes) + " Gigabyte. Ploting will take several seconds.</b>" 
            Warning_HTML.visible=True            
        else:
            Warning_HTML.visible=False



def MainTab_Changed( change ):
    if change['type']=='change' and change['name']=='selected_index':
        if change['new'] == 0:
            change = {'type':'change', 'name':'value', 'new':tiegcmFolder_Dropdown.value}
        else:
            change = {'type':'change', 'name':'value', 'new':SavedFilenames_Dropdown.value}
        tiegcmFolder_Dropdown_onChange( change )  
            
def createGUI():
    ## the top level visual elements
    MainPanel = w.VBox()    
    MainTab = w.Tab() 
    LoadCoveragePanel           = w.VBox()
    CalcCoveragePanel           = w.VBox()
    CalcCoverageAlongOrbitPanel = w.VBox()
    CompareResultsPanel         = w.VBox()
    ## the checkboxes which allow user to select which plots he wants to create
    PlotSelectionPanel = w.VBox()
    PlotSelectionPanel.children = [Plot_JHvsMagLat_Checkbox, Plot_JHvsMLT_Checkbox, Plot_JHvsAltitude_Checkbox, Plot_AltitudeVsMagLat_Checkbox, w.HBox([Plot_JHdistribution_Checkbox, RegressionOptions_Dropdown]) ]
    ##
    MainTab.children = [ CalcCoverageAlongOrbitPanel, CalcCoveragePanel, LoadCoveragePanel, CompareResultsPanel ]
    MainTab.set_title(0, 'Calc JH along Orbit')
    MainTab.set_title(1, 'Calculate JH-per-Bin')
    MainTab.set_title(2, 'Load JH-per-Bin')
    MainTab.set_title(3, 'Compare Results')
    MainPanel.children = [ MainTab, OrbitPreviewImage ]    
    ## 
    Exec_Btn = w.Button (description='Calculate JH-per-Bin',tooltip="Click here to calculate",)
    Exec_Btn.style.button_color = 'MediumTurquoise'
    Exec_Btn.on_click( Exec_Btn_Clicked )
    CalcCoveragePanel.children = [tiegcmFolder_Dropdown, BinGroups_Dropdown, ExecutionTitle_Text, ExecutionDescr_Text, Exec_Btn ] # I removed PlotSelectionPanel
    ##
    ExecAlongOrbit_Btn = w.Button (description='Calc JH along Orbit',tooltip="Click here to calculate",)
    ExecAlongOrbit_Btn.style.button_color = 'Coral'
    ExecAlongOrbit_Btn.on_click( Exec_Btn_alongOrbit_Clicked )
    CalcCoverageAlongOrbitPanel.children = [tiegcmFolder_Dropdown, BinGroups_Dropdown, OrbitFilename_Dropdown, ExecutionTitle_Text, ExecutionDescr_Text, ExecAlongOrbit_Btn ] # I removed PlotSelectionPanel
    ##
    Load_Btn = w.Button (description='Load JH-per-Bin from',tooltip="Click here to plot",)
    Load_Btn.style.button_color = 'YellowGreen'
    Load_Btn.on_click( Load_Btn_Clicked )
    L2_horizontal = w.HBox()
    L2_horizontal.children = [Load_Btn, SavedFilenames_Dropdown]
    LoadCoveragePanel.children = [ w.HBox([Load_Btn, SavedFilenames_Dropdown]), Warning_HTML, PlotSelectionPanel ]
    ##
    CompareResults_Btn = w.Button (description='Compare Results',tooltip="Click here to plot",)
    CompareResults_Btn.style.button_color = 'Gold'
    CompareResults_Btn.on_click( CompareResults_Btn_Clicked )
    CompareResultsPanel.children = [ w.HBox([w.HTML(value="TIEGCM results:"),SavedFilenames_Dropdown]), w.HBox([w.HTML(value="Orbit results:"),SavedFilenames2_Dropdown]), CompareResults_Btn ]
    ## Assign event listeners
    tiegcmFolder_Dropdown.observe( tiegcmFolder_Dropdown_onChange )
    SavedFilenames_Dropdown.observe( SavedFilenames_Dropdown_onChange )
    MainTab.observe( MainTab_Changed )
    ## display orbit-related image
    change = {'type':'change', 'name':'value', 'new':tiegcmFolder_Dropdown.value}
    tiegcmFolder_Dropdown_onChange( change )        
    return MainPanel
# PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT 
# PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT 
# PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT PLOT 

def plotAll():
    global all_JH_values, all_MagLat_values, all_MLT_values, all_Altitude_values
    
    # choose which bins we are going to work with
    RegionID = CALCULATIONS_RegionName[ CALCULATIONS_RegionName.find('(')+1 : CALCULATIONS_RegionName.rfind(')') ]
    BinsIncludedAtPlot = list()
    for B in Bins:
        if B.ID.startswith(RegionID): BinsIncludedAtPlot.append( B )

    # remember the Kp ranges of these bins. Each Kp-range will have its own sub-plot
    All_KpRanges = list()
    for B in BinsIncludedAtPlot:
        if [B.Kp_min, B.Kp_max] not in All_KpRanges: All_KpRanges.append( [B.Kp_min, B.Kp_max] )  
            
    # --- init various plotting parameters ---
    max_num_of_points = 10000
    plot_step = int(  len(all_JH_values) / max_num_of_points  )
    if plot_step <= 0: plot_step = 1
    print( "I will plot", max_num_of_points, "out of", len(all_JH_values), "points (1 per", plot_step, ")")
    all_JH_values = all_JH_values[0::plot_step]
    all_MagLat_values = all_MagLat_values[0::plot_step]
    all_MLT_values = all_MLT_values[0::plot_step]
    all_Altitude_values = all_Altitude_values[0::plot_step]
    # handle MLT ranges like 22:00-02:00
    MLT_min_toPlot = BinsIncludedAtPlot[0].MLT_min
    MLT_max_toPlot = BinsIncludedAtPlot[0].MLT_max
    if BinsIncludedAtPlot[0].MLT_min > BinsIncludedAtPlot[0].MLT_max:
        MLT_max_toPlot += 24
        for i in range(0, len(all_MLT_values)):
            if all_MLT_values[i] < BinsIncludedAtPlot[0].MLT_min: all_MLT_values[i] += 24
    # define altitude range for X-axis
    Altitude_max_toPlot = max(all_Altitude_values)
    if Altitude_max_toPlot < 140: Altitude_max_toPlot = 140
                
    # define max JH value to be plotted
    JHmax = 1.2e-7
    if CALCULATIONS_RegionName.startswith( "SQ" ): JHmax = max(all_JH_values)
    
    if len(all_MagLat_values) > 0  and  Plot_JHvsMagLat_Checkbox.value==True:
        print( "Plotting ", len(all_MagLat_values), "points" )
        MyColorsIndex = 0
        fig = go.Figure()        
        fig.add_trace( go.Scatter(name="JH", x=all_MagLat_values, y=all_JH_values, mode='markers', marker_size=2) )
        BinAnnotations = list()
        prevKpMin = -1
        BinIdx = 0
        for B in BinsIncludedAtPlot:
            if len(B.JH_values) > 0:
                # choose color for mean line
                if prevKpMin >= 0 and prevKpMin != B.Kp_min:
                    MyColorsIndex += 1
                    if MyColorsIndex>len(MyColors)-1: MyColorsIndex = 0
                prevKpMin = B.Kp_min                        
                # add visuals for the mean line
                fig.add_shape( type="line", x0=B.MagLat_min, y0=B.JH_mean,     x1=B.MagLat_max, y1=B.JH_mean,     line=dict( color=MyColors[MyColorsIndex], width=2, ), )    
                # add info as legend for this bin
                fig.add_trace( go.Scatter(name=B.ID + ":  " + str(B.Altitude_min) + "<Alt<"+ str(B.Altitude_max) + "  <b>" + str(B.Kp_min) + "<Kp<" + str(B.Kp_max) + "</b>" + "  Mean=" + "{:.3e}".format(B.JH_mean) + "  " + "Variance=" + "{:.3e}".format(B.JH_variance) + "  St.Deviation=" + "{:.3e}".format(B.JH_variance**(1/2)) , x=[-1], y=[-1], mode='markers', marker_size=1, marker_color=MyColors[MyColorsIndex]) )
                # add bin name above the mean line
                BinAnnotations.append( dict( x=B.MagLat_min+((BinIdx+1)/len(BinsIncludedAtPlot))*(B.MagLat_max-B.MagLat_min)*3/4, y=B.JH_mean, xref="x", yref="y", text=B.ID, showarrow=False, yshift=8, font=dict(color=MyColors[MyColorsIndex])) )
                # add visuals for standard deviation
                fig.add_shape( type="line", x0=B.MagLat_min+((BinIdx+1)/len(BinsIncludedAtPlot))*(B.MagLat_max-B.MagLat_min)*7/8, y0=B.JH_mean+(B.JH_variance)**(1/2)/2,     x1=B.MagLat_min+((BinIdx+1)/len(BinsIncludedAtPlot))*(B.MagLat_max-B.MagLat_min)*7/8, y1=B.JH_mean-(B.JH_variance)**(1/2)/2,     line=dict( color=MyColors[MyColorsIndex], width=1, ), ) 
                #
                BinIdx += 1
        fig.update_layout( annotations=BinAnnotations )
        fig.update_layout( title="JH vs Magnetic Latitude - " + CALCULATIONS_RegionName, 
                           width=1000, height=1300, legend_orientation="h", legend= {'itemsizing': 'constant'}) 
        fig.update_xaxes(range=[min(all_MagLat_values), max(all_MagLat_values)], title="Magnetic Latitude (degrees)")
        fig.update_yaxes(range=[min(all_JH_values), JHmax], title="Joule Heating (W/m3)", showexponent = 'all', exponentformat = 'e')
        plotly.offline.init_notebook_mode(connected=True)
        plotly.offline.iplot(fig)
    else:
        print( "There are no points for MagLat plot" )                

    if len(all_MLT_values) > 0  and  Plot_JHvsMLT_Checkbox.value == True:
        MyColorsIndex = 0
        fig = go.Figure()
        print( "Plotting ", len(all_MLT_values), "points" )
        fig.add_trace( go.Scatter(name="JH", x=all_MLT_values, y=all_JH_values, mode='markers', marker_size=2) )
        prevKpMin = -1
        BinAnnotations = list()
        BinIdx = 0
        for B in BinsIncludedAtPlot:
            if len(B.JH_values) > 0:
                # choose color for mean line
                if prevKpMin >= 0 and prevKpMin != B.Kp_min:
                    MyColorsIndex += 1
                    if MyColorsIndex>len(MyColors)-1: MyColorsIndex = 0
                prevKpMin = B.Kp_min                        
                # add visuals for the mean line             
                fig.add_shape( type="line", x0=MLT_min_toPlot, y0=B.JH_mean,     x1=MLT_max_toPlot, y1=B.JH_mean,     line=dict( color=MyColors[MyColorsIndex], width=2, ), )    
                # add info as legend for this bin
                fig.add_trace( go.Scatter(name=B.ID + ":  " + str(B.Altitude_min) + "<Alt<"+ str(B.Altitude_max) + "  <b>" + str(B.Kp_min) + "<Kp<" + str(B.Kp_max) + "</b>" + "  Mean=" + "{:.3e}".format(B.JH_mean) + "  " + "Variance=" + "{:.3e}".format(B.JH_variance) + "St.Deviation=" + "{:.3e}".format(B.JH_variance**(1/2)), x=[-1], y=[-1], mode='markers', marker_size=1, marker_color=MyColors[MyColorsIndex]) )
                # add bin name above the mean line
                BinAnnotations.append( dict( x=MLT_min_toPlot+((BinIdx+1)/len(BinsIncludedAtPlot))*(MLT_max_toPlot-MLT_min_toPlot)*3/4, y=B.JH_mean, xref="x", yref="y", text=B.ID, showarrow=False, yshift=8, font=dict(color=MyColors[MyColorsIndex])) )
                # add visuals for standard deviation
                fig.add_shape( type="line", x0=MLT_min_toPlot+((BinIdx+1)/len(BinsIncludedAtPlot))*(MLT_max_toPlot-MLT_min_toPlot)*7/8, y0=B.JH_mean+(B.JH_variance)**(1/2)/2,     x1=MLT_min_toPlot+((BinIdx+1)/len(BinsIncludedAtPlot))*(MLT_max_toPlot-MLT_min_toPlot)*7/8, y1=B.JH_mean-(B.JH_variance)**(1/2)/2,     line=dict( color=MyColors[MyColorsIndex], width=1, ), )
                #
                BinIdx += 1
        fig.update_layout( annotations=BinAnnotations )
        fig.update_layout( title="JH vs Magnetic Local Time - " + CALCULATIONS_RegionName, 
                           width=1000, height=1300, legend_orientation="h", legend= {'itemsizing': 'constant'}) 
        fig.update_xaxes(range=[MLT_min_toPlot, MLT_max_toPlot], title="Magnetic Local Time (hours)") #fig.update_xaxes(range=[min(all_MLT_values), max(all_MLT_values)], title="Magnetic Local Time (hours)")
        fig.update_yaxes(range=[min(all_JH_values), JHmax], title="Joule Heating (W/m3)", showexponent = 'all', exponentformat = 'e')
        plotly.offline.init_notebook_mode(connected=True)
        plotly.offline.iplot(fig)
    else:
        print( "There are no points for MLT plot" )        
    
    if len(all_Altitude_values) > 0  and  Plot_JHvsAltitude_Checkbox.value == True:
        MyColorsIndex = 0
        fig = go.Figure()
        print( "Plotting ", len(all_Altitude_values), "points" )
        fig.add_trace( go.Scatter(name="JH", x=all_Altitude_values, y=all_JH_values, mode='markers', marker_size=2) )
        prevKpMin = -1
        BinAnnotations = list()
        BinIdx = 0
        for B in BinsIncludedAtPlot:
            if len(B.JH_values) > 0:
                # choose color for mean line
                if prevKpMin >= 0 and prevKpMin != B.Kp_min:
                    MyColorsIndex += 1
                    if MyColorsIndex>len(MyColors)-1: MyColorsIndex = 0
                prevKpMin = B.Kp_min                        
                # add visuals for the mean line
                fig.add_shape( type="line", x0=B.Altitude_min, y0=B.JH_mean,     x1=B.Altitude_max, y1=B.JH_mean,     line=dict( color=MyColors[MyColorsIndex], width=2, ), )    
                # add info as legend for this bin
                fig.add_trace( go.Scatter(name=B.ID + ":  " + str(B.Altitude_min) + "<Alt<"+ str(B.Altitude_max) + "  <b>" + str(B.Kp_min) + "<Kp<" + str(B.Kp_max) + "</b>" + "  Mean=" + "{:.3e}".format(B.JH_mean) + "  " + "Variance=" + "{:.3e}".format(B.JH_variance) + "  St.Deviation=" + "{:.3e}".format(B.JH_variance**(1/2)), x=[-1], y=[-1], mode='markers', marker_size=1, marker_color=MyColors[MyColorsIndex]) )
                # add bin name above the mean line
                BinAnnotations.append( dict( x=B.Altitude_min+((BinIdx+1)/len(BinsIncludedAtPlot))*(B.Altitude_max-B.Altitude_min)*3/4, y=B.JH_mean, xref="x", yref="y", text=B.ID, showarrow=False, yshift=8, font=dict(color=MyColors[MyColorsIndex])) )
                # add visuals for standard deviation
                fig.add_shape( type="line", x0=B.Altitude_min+((BinIdx+1)/len(BinsIncludedAtPlot))*(B.Altitude_max-B.Altitude_min)*7/8, y0=B.JH_mean+(B.JH_variance)**(1/2)/2,     x1=B.Altitude_min+((BinIdx+1)/len(BinsIncludedAtPlot))*(B.Altitude_max-B.Altitude_min)*7/8, y1=B.JH_mean-(B.JH_variance)**(1/2)/2,     line=dict( color=MyColors[MyColorsIndex], width=1, ), )
                #
                BinIdx += 1
        fig.update_layout( annotations=BinAnnotations )
        fig.update_layout( title="JH vs Altitude - " + CALCULATIONS_RegionName, 
                           width=1000, height=1300, legend_orientation="h", legend= {'itemsizing': 'constant'}) 
        fig.update_xaxes(range=[115, Altitude_max_toPlot], title="Altitude (km)")
        fig.update_yaxes(range=[min(all_JH_values), JHmax], title="Joule Heating (W/m3)", showexponent = 'all', exponentformat = 'e')
        plotly.offline.init_notebook_mode(connected=True)
        plotly.offline.iplot(fig)
    else:
        print( "There are no points for Altitude plot" )
    
    
    if len(all_JH_values) > 0  and  Plot_AltitudeVsMagLat_Checkbox.value == True:
        MyColorsIndex = 0
        fig = go.Figure()
        print( "Plotting ", len(all_JH_values), "points" )
        
        colorMean = 0
        for n in all_JH_values: colorMean += n
        colorMean = colorMean / len( all_JH_values )
        colorMin = colorMean / 10
        colorMax = colorMean * 10
        fig.add_trace( go.Scatter(name="JH", x=all_MagLat_values, y=all_Altitude_values, mode='markers', 
                       marker=dict( size=2, color=all_JH_values, colorscale="Jet", cmin=colorMin, cmax=colorMax, colorbar=dict(title="JH (W/m3)" )) ) )
        for B in BinsIncludedAtPlot:
            if len(B.JH_values) > 0:
                #fig.add_shape( type="line", x0=B.MagLat_min, y0=B.JH_mean,     x1=B.MagLat_max, y1=B.JH_mean,     line=dict( color=MyColors[MyColorsIndex], width=1, ), )    
                #fig.add_trace( go.Scatter(name="Bin Mean: " + str(B.Altitude_min) + "<Alt<"+ str(B.Altitude_max) + " <b>" + str(B.Kp_min) + "<Kp<" + str(B.Kp_max) + "</b> Variance=" + str(B.JH_variance), x=[-1], y=[-1], mode='markers', marker_size=1, marker_color=MyColors[MyColorsIndex]) )
                MyColorsIndex += 1
                if MyColorsIndex>len(MyColors)-1: MyColorsIndex = 0
        fig.update_layout( title="Altitude vs Magnetic Latitude - " + CALCULATIONS_RegionName, 
                           width=1000, height=1300, legend_orientation="h", legend= {'itemsizing': 'constant'}) 
        fig.update_xaxes(range=[min(all_MagLat_values), max(all_MagLat_values)], title="Magnetic Latitude (degrees)" )
        fig.update_yaxes(range=[min(all_Altitude_values), max(all_Altitude_values)], title="Altitude(km)")
        plotly.offline.init_notebook_mode(connected=True)
        plotly.offline.iplot(fig)
    else:
        print( "There are no points for Altitude-MagLat plot" )                

    

# PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP 
# PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP 
# PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP PERKP 

def plotAll_perKp():
    global all_JH_values, all_MagLat_values, all_MLT_values, all_Altitude_values
    
    # choose which bins we are going to work with
    RegionID = CALCULATIONS_RegionName[ CALCULATIONS_RegionName.find('(')+1 : CALCULATIONS_RegionName.rfind(')') ]
    BinsIncludedAtPlot = list()
    for B in Bins:
        if B.ID.startswith(RegionID): BinsIncludedAtPlot.append( B )          

    # --- init various plotting parameters ---
    max_num_of_points = 4000
    # handle MLT ranges like 22:00-02:00
    MLT_min_toPlot = BinsIncludedAtPlot[0].MLT_min
    MLT_max_toPlot = BinsIncludedAtPlot[0].MLT_max
    if BinsIncludedAtPlot[0].MLT_min > BinsIncludedAtPlot[0].MLT_max:
        MLT_max_toPlot += 24
        for i in range(0, len(all_MLT_values)):
            if all_MLT_values[i] < BinsIncludedAtPlot[0].MLT_min: all_MLT_values[i] += 24
        for B in BinsIncludedAtPlot:
            for i in range(0, len(B.MLT_values)):
                if B.MLT_values[i] < BinsIncludedAtPlot[0].MLT_min: B.MLT_values[i] += 24
    # define altitude range for X-axis
    Altitude_max_toPlot = max(all_Altitude_values)
    if Altitude_max_toPlot < 140: Altitude_max_toPlot = 140
    # define max JH value to be plotted
    JHmax = 1.2e-7
    if CALCULATIONS_RegionName.startswith( "SQ" ): JHmax = max(all_JH_values)


    # CONSTRUCT DATA per Kp-range
    # remember the Kp ranges of the plot's bins. Each Kp-range will have its own sub-plot
    All_KpRanges = list()
    for B in BinsIncludedAtPlot:
        if [B.Kp_min, B.Kp_max] not in All_KpRanges: 
            All_KpRanges.append( [B.Kp_min, B.Kp_max] )  
    # group data according to Kp-range
    JH_values_perKp       = list() # 2d-list: one row for each Kp-range
    MagLat_values_perKp   = list() # 2d-list: one row for each Kp-range
    MLT_values_perKp      = list() # 2d-list: one row for each Kp-range
    Altitude_values_perKp = list() # 2d-list: one row for each Kp-range
    for i in range(0, len(All_KpRanges)): 
        JH_values_perKp.append( list() )
        MagLat_values_perKp.append( list() )
        MLT_values_perKp.append( list() )
        Altitude_values_perKp.append( list() )
        for B in BinsIncludedAtPlot:
            if B.Kp_min==All_KpRanges[i][0] and B.Kp_max==All_KpRanges[i][1]: 
                JH_values_perKp[i]       += B.JH_values
                MagLat_values_perKp[i]   += B.MagLat_values
                MLT_values_perKp[i]      += B.MLT_values
                Altitude_values_perKp[i] += B.Altitude_values    
    # make the data set smaller so that it can be plotted
    print( "\n" ) 
    for i in range(0, len(All_KpRanges)):
        plot_step = int(  len(JH_values_perKp[i]) / max_num_of_points  )
        print( "I will plot", max_num_of_points, "out of", len(JH_values_perKp[i]), "points (1 per", plot_step, ")" + " for " + str(All_KpRanges[i][0]) + "<Kp<" + str(All_KpRanges[i][1]) )
        if plot_step > 0:
            JH_values_perKp[i]       = JH_values_perKp[i][0::plot_step]
            MagLat_values_perKp[i]   = MagLat_values_perKp[i][0::plot_step]
            MLT_values_perKp[i]      = MLT_values_perKp[i][0::plot_step]
            Altitude_values_perKp[i] = Altitude_values_perKp[i][0::plot_step]   
            
    # PLOT
    if len(all_MagLat_values) > 0  and  Plot_JHvsMagLat_Checkbox.value==True:
        fig = make_subplots(rows=len(All_KpRanges), cols=1, shared_xaxes=False, vertical_spacing=0.05)
        for i in range(0, len(All_KpRanges)):
            fig.append_trace( go.Scatter(name="JH", x=MagLat_values_perKp[i], y=JH_values_perKp[i], mode='markers', marker_size=2, marker_color=MyColors[i]), row=i+1, col=1 )
        #
        BinAnnotations = list()
        FigureShapes = list()
        MyColorsIndex = 0
        BinIdx = 0
        for B in BinsIncludedAtPlot:
            if len(B.JH_values) > 0:
                # choose which sub-plot will host this Bin's data
                SubPlotIdx = 1
                for i in range(0, len(All_KpRanges)):
                    if B.Kp_min==All_KpRanges[i][0] and B.Kp_max==All_KpRanges[i][1]: SubPlotIdx = i+1
                # choose color for mean line
                MyColorsIndex = SubPlotIdx - 1
                # add visuals for the mean line
                #fig.append_shape( dict(type="line", x0=B.MagLat_min, y0=B.JH_mean,     x1=B.MagLat_max, y1=B.JH_mean,   line=dict( color=MyColors[MyColorsIndex], width=2, )), row=SubPlotIdx, col=1 )    
                #print( fig["layout"] )
                #print( type (fig["layout"].shapes) )
                FigureShapes.append( dict(type="line", x0=B.MagLat_min, y0=B.JH_mean,     x1=B.MagLat_max, y1=B.JH_mean,   line=dict( color=MyColors[MyColorsIndex], width=2, ),  xref= 'x'+str(SubPlotIdx), yref= 'y'+str(SubPlotIdx))  )                
                # add info as legend for this bin
                fig.append_trace( go.Scatter(name=B.ID + ":  " + str(B.Altitude_min) + "<Alt<"+ str(B.Altitude_max) + "  <b>" + str(B.Kp_min) + "<Kp<" + str(B.Kp_max) + "</b>" + "  Mean=" + "{:.3e}".format(B.JH_mean) + "  " + "Variance=" + "{:.3e}".format(B.JH_variance) + "  St.Deviation=" + "{:.3e}".format(B.JH_variance**(1/2)), x=[-1], y=[-1], mode='markers', marker_size=1, marker_color=MyColors[MyColorsIndex]), row=SubPlotIdx, col=1 )
                # add bin name above the mean line
                BinAnnotations.append( dict( x=B.MagLat_min+((BinIdx+1)/len(BinsIncludedAtPlot))*(B.MagLat_max-B.MagLat_min)*3/4, y=B.JH_mean, text=B.ID, showarrow=False, yshift=8, font=dict(color=MyColors[MyColorsIndex]), xref='x1', yref='y'+str(SubPlotIdx) ) )
                # add visuals for standard deviation
                FigureShapes.append( dict(type="line", x0=B.MagLat_min+((BinIdx+1)/len(BinsIncludedAtPlot))*(B.MagLat_max-B.MagLat_min)*7/8, y0=B.JH_mean+(B.JH_variance)**(1/2)/2,     x1=B.MagLat_min+((BinIdx+1)/len(BinsIncludedAtPlot))*(B.MagLat_max-B.MagLat_min)*7/8, y1=B.JH_mean-(B.JH_variance)**(1/2)/2,     line=dict( color=MyColors[MyColorsIndex], width=2, ), xref= 'x1', yref='y'+str(SubPlotIdx) )  )
                #
                BinIdx += 1
        fig.update_layout( annotations=BinAnnotations )
        fig.update_layout(shapes=FigureShapes)
        fig.update_layout( title="JH vs Magnetic Latitude - " + CALCULATIONS_RegionName, 
                           width=1000, height=1500, legend_orientation="h", legend= {'itemsizing': 'constant'}) 
        #fig.update_xaxes(range=[min(all_MagLat_values), max(all_MagLat_values)])
        #for i in range(0, len(All_KpRanges)):
        #fig.update_xaxes(range=[min(all_MagLat_values), max(all_MagLat_values)], title="Magnetic Latitude (degrees) for " + str(All_KpRanges[i][0]) + "<Kp<"+  str(All_KpRanges[i][1]), row=SubPlotIdx, col=1 )
        for i in range(0, len(All_KpRanges)):
            fig.update_xaxes(range=[min(all_MagLat_values), max(all_MagLat_values)], title="Magnetic Latitude (degrees) for " + str(All_KpRanges[i][0]) + "<Kp<"+  str(All_KpRanges[i][1]), row=i+1, col=1 )
        fig.update_yaxes(range=[min(all_JH_values), JHmax], title="Joule Heating (W/m3)", showexponent = 'all', exponentformat = 'e')
        plotly.offline.init_notebook_mode(connected=True)
        plotly.offline.iplot(fig)
    else:
        print( "There are no points for MagLat per-Kp-range plot" )                
    
    ##
    if len(all_MLT_values) > 0  and  Plot_JHvsMLT_Checkbox.value == True:
        fig = make_subplots(rows=len(All_KpRanges), cols=1, shared_xaxes=False, vertical_spacing=0.05)
        for i in range(0, len(All_KpRanges)):
            fig.append_trace( go.Scatter(name="JH", x=MLT_values_perKp[i], y=JH_values_perKp[i], mode='markers', marker_size=2, marker_color=MyColors[i]), row=i+1, col=1 )
        #
        BinAnnotations = list()
        FigureShapes = list()
        MyColorsIndex = 0
        BinIdx = 0
        for B in BinsIncludedAtPlot:
            if len(B.JH_values) > 0:
                # choose which sub-plot will host this Bin's data
                SubPlotIdx = 1
                for i in range(0, len(All_KpRanges)):
                    if B.Kp_min==All_KpRanges[i][0] and B.Kp_max==All_KpRanges[i][1]: SubPlotIdx = i+1
                    #fig.update_xaxes(range=[min(MLT_values_perKp), max(MLT_values_perKp)], title="Magnetic Local Time (hours) for " + str(All_KpRanges[i][0]) + "<Kp<"+  str(All_KpRanges[i][1]), row=SubPlotIdx, col=1 )                
                # choose color for mean line
                MyColorsIndex = SubPlotIdx - 1
                # add visuals for the mean line             
                FigureShapes.append( dict(type="line", x0=MLT_min_toPlot, y0=B.JH_mean,     x1=MLT_max_toPlot, y1=B.JH_mean,   line=dict( color=MyColors[MyColorsIndex], width=2, ),  xref= 'x'+str(SubPlotIdx), yref= 'y'+str(SubPlotIdx))  ) 
                # add info as legend for this bin
                fig.append_trace( go.Scatter(name=B.ID + ":  " + str(B.Altitude_min) + "<Alt<"+ str(B.Altitude_max) + "  <b>" + str(B.Kp_min) + "<Kp<" + str(B.Kp_max) + "</b>" + "  Mean=" + "{:.3e}".format(B.JH_mean) + "  " + "Variance=" + "{:.3e}".format(B.JH_variance) + "  St.Deviation=" + "{:.3e}".format(B.JH_variance**(1/2)), x=[-1], y=[-1], mode='markers', marker_size=1, marker_color=MyColors[MyColorsIndex]), row=SubPlotIdx, col=1 )
                # add bin name above the mean line
                BinAnnotations.append(          dict( x=MLT_min_toPlot+((BinIdx+1)/len(BinsIncludedAtPlot))*(MLT_max_toPlot-MLT_min_toPlot)*3/4, y=B.JH_mean, text=B.ID, showarrow=False, yshift=8, font=dict(color=MyColors[MyColorsIndex]), xref='x1', yref='y'+str(SubPlotIdx)) )
                FigureShapes.append( dict(type="line", x0=MLT_min_toPlot+((BinIdx+1)/len(BinsIncludedAtPlot))*(MLT_max_toPlot-MLT_min_toPlot)*7/8, y0=B.JH_mean+(B.JH_variance)**(1/2)/2,     x1=MLT_min_toPlot+((BinIdx+1)/len(BinsIncludedAtPlot))*(MLT_max_toPlot-MLT_min_toPlot)*7/8, y1=B.JH_mean-(B.JH_variance)**(1/2)/2,     line=dict( color=MyColors[MyColorsIndex], width=2, ), xref= 'x1', yref= 'y'+str(SubPlotIdx) )  )
                #
                BinIdx += 1
        fig.update_layout( annotations=BinAnnotations )
        fig.update_layout( shapes=FigureShapes )
        fig.update_layout( title="JH vs Magnetic Local Time - " + CALCULATIONS_RegionName, 
                           width=1000, height=1500, legend_orientation="h", legend= {'itemsizing': 'constant'}) 
        #fig.update_xaxes(range=[MLT_min_toPlot, MLT_max_toPlot], title="Magnetic Local Time (hours)")
        for i in range(0, len(All_KpRanges)):
            fig.update_xaxes(range=[MLT_min_toPlot, MLT_max_toPlot], title="Magnetic Local Time (hours) for " + str(All_KpRanges[i][0]) + "<Kp<"+  str(All_KpRanges[i][1]), row=i+1, col=1 )
        fig.update_yaxes(range=[min(all_JH_values), JHmax], title="Joule Heating (W/m3)", showexponent = 'all', exponentformat = 'e')
        plotly.offline.init_notebook_mode(connected=True)
        plotly.offline.iplot(fig)
    else:
        print( "There are no points for MLT per-Kp-range plot" )        
    
    if len(all_Altitude_values) > 0  and  Plot_JHvsAltitude_Checkbox.value == True:
        fig = make_subplots(rows=len(All_KpRanges), cols=1, shared_xaxes=False, vertical_spacing=0.05)
        for i in range(0, len(All_KpRanges)):
            fig.append_trace( go.Scatter(name="JH", x=Altitude_values_perKp[i], y=JH_values_perKp[i], mode='markers', marker_size=2, marker_color=MyColors[i]), row=i+1, col=1 )
        #
        BinAnnotations = list()
        FigureShapes = list()
        MyColorsIndex = 0
        BinIdx = 0
        for B in BinsIncludedAtPlot:
            if len(B.JH_values) > 0:
                # choose which sub-plot will host this Bin's data
                SubPlotIdx = 1
                for i in range(0, len(All_KpRanges)):
                    if B.Kp_min==All_KpRanges[i][0] and B.Kp_max==All_KpRanges[i][1]: SubPlotIdx = i+1
                    fig.update_xaxes(range=[115, Altitude_max_toPlot], title="Altitude (km) for " + str(All_KpRanges[i][0]) + "<Kp<"+  str(All_KpRanges[i][1]), row=SubPlotIdx, col=1 )
                # choose color for mean line
                MyColorsIndex = SubPlotIdx - 1                
                # add visuals for the mean line
                FigureShapes.append( dict(type="line", x0=B.Altitude_min, y0=B.JH_mean,     x1=B.Altitude_max, y1=B.JH_mean,   line=dict( color=MyColors[MyColorsIndex], width=2, ),  xref= 'x'+str(SubPlotIdx), yref= 'y'+str(SubPlotIdx))  )                
                # add info as legend for this bin
                fig.append_trace( go.Scatter(name=B.ID + ":  " + str(B.Altitude_min) + "<Alt<"+ str(B.Altitude_max) + "  <b>" + str(B.Kp_min) + "<Kp<" + str(B.Kp_max) + "</b>" + "  Mean=" + "{:.3e}".format(B.JH_mean) + "  " + "Variance=" + "{:.3e}".format(B.JH_variance) + "  St.Deviation=" + "{:.3e}".format(B.JH_variance**(1/2)), x=[-1], y=[-1], mode='markers', marker_size=1, marker_color=MyColors[MyColorsIndex]), row=SubPlotIdx, col=1 )
                # add bin name above the mean line
                BinAnnotations.append( dict( x=B.Altitude_min+((BinIdx+1)/len(BinsIncludedAtPlot))*(B.Altitude_max-B.Altitude_min)*3/4, y=B.JH_mean, text=B.ID, showarrow=False, yshift=8, font=dict(color=MyColors[MyColorsIndex]), xref='x1', yref='y'+str(SubPlotIdx) ) )
                # add visuals for standard deviation
                FigureShapes.append( dict(type="line", x0=B.Altitude_min+((BinIdx+1)/len(BinsIncludedAtPlot))*(B.Altitude_max-B.Altitude_min)*7/8, y0=B.JH_mean+(B.JH_variance)**(1/2)/2,     x1=B.Altitude_min+((BinIdx+1)/len(BinsIncludedAtPlot))*(B.Altitude_max-B.Altitude_min)*7/8, y1=B.JH_mean-(B.JH_variance)**(1/2)/2,     line=dict( color=MyColors[MyColorsIndex], width=2, ), xref= 'x1', yref= 'y'+str(SubPlotIdx) )  )
                #
                BinIdx += 1
        fig.update_layout( annotations=BinAnnotations )
        fig.update_layout( shapes=FigureShapes )
        fig.update_layout( title="JH vs Altitude - " + CALCULATIONS_RegionName, 
                           width=1000, height=1500, legend_orientation="h", legend= {'itemsizing': 'constant'}) 
        #fig.update_xaxes(range=[115, Altitude_max_toPlot], title="Altitude (km)")
        fig.update_yaxes(range=[min(all_JH_values), JHmax], title="Joule Heating (W/m3)", showexponent = 'all', exponentformat = 'e')
        plotly.offline.init_notebook_mode(connected=True)
        plotly.offline.iplot(fig)
    else:
        print( "There are no points for Altitude per-Kp-range plot" )
    
    

    
    
    
    
    
# Y = a*X^k + c
def func_powerlaw(x,  a, k, c):
    return a * (x**k)  +  c

# Y = a * log(X) + c
def func_logarithmic(x,  a, c):
    return [ (a * math.log(x_i) + c)  for x_i in x ]

# Y = a / e ^ (bx) + c
def func_euler(x,  a, b, c):
    return [ (a / (math.e**(b*x_i)) + c) for x_i in x ]

def func_maxwellian(x,  a, b, c):
    return [ (a * x_i*x_i * (math.e**(-b*x_i)) + c) for x_i in x ]

def Plot_JH_Distribution_perBin():
    # choose which bins we are going to work with
    RegionID = CALCULATIONS_RegionName[ CALCULATIONS_RegionName.find('(')+1 : CALCULATIONS_RegionName.rfind(')') ]
    BinsIncludedAtPlot = list()
    for B in Bins:
        if B.ID.startswith(RegionID): BinsIncludedAtPlot.append( B )
    
    # init 
    upper_value = (2)*BinsIncludedAtPlot[0].JH_mean
    lower_value = 0
    slot_length = (upper_value - lower_value) / 100  
    if slot_length == 0: 
        print( "No values for Distribution Plot" )
        return
    
    # calculate JH distribution for each bin
    for B in BinsIncludedAtPlot:
        print(B.ID, "distribution:")
        for aJHval in B.JH_values:
            if aJHval == upper_value:
                slot_idx = 100 - 1            
            elif aJHval < upper_value:
                slot_idx = int(   (aJHval - lower_value) / slot_length  )
            else:
                continue
            B.JH_distribution[ slot_idx ] += 1;
        print(B.JH_distribution, "\n")
    
    
    # plot the distribution of all bins on the same figure
    if len(all_JH_values) > 0  and  Plot_JHdistribution_Checkbox.value == True:
        MyColorsIndex = 0
        prevKpMin = -1
        BinAnnotations = list()
        BinIdx = 0
        fig = go.Figure()        
        print( "Plotting JH Distribution"  )
        for B in BinsIncludedAtPlot:
            if len(B.JH_values) > 0:
                # choose color for this bin's points
                if prevKpMin >= 0 and prevKpMin != B.Kp_min:
                    MyColorsIndex += 1
                    if MyColorsIndex>len(MyColors)-1: MyColorsIndex = 0
                prevKpMin = B.Kp_min                        

                if RegressionOptions_Dropdown.value.startswith( "Polynomial" ):
                    # calculate the Polynomial Regression
                    degree = int( RegressionOptions_Dropdown.value[-1] )
                    myPolynomial = np.polyfit( list(range(0,100)), B.JH_distribution, degree )
                    # construct the equation to display
                    poly_str = "y = "
                    for i in range(0, len(myPolynomial)): 
                        if i>0 and myPolynomial[i] > 0: poly_str += "+ "
                        poly_str += "{:.2e}".format(myPolynomial[i])
                        if i < len(myPolynomial)-1: poly_str += "x^" + str(len(myPolynomial)-1-i) + " "
                    # draw the Polynomial Regression
                    mymodel = np.poly1d(myPolynomial)
                    myline = np.linspace(1, 100, 100)
                    fig.add_trace( go.Scatter(name=B.ID+":  "+poly_str, x=myline, y=mymodel(myline), line=dict(color=MyColors[MyColorsIndex], width=1) ) )
                elif RegressionOptions_Dropdown.value == "Power law":
                    OptimalParams, OptParamsCovariance = curve_fit(func_powerlaw, list(range(0,100)), B.JH_distribution)
                    poly_str = "y = " + "{:.2e}".format(OptimalParams[0]) + " * x^" + "{:.2e}".format(OptimalParams[1]) + " + " + "{:.2e}".format(OptimalParams[2])
                    fig.add_trace( go.Scatter(name=B.ID+":  "+poly_str, x=list(range(0,100)), y=func_powerlaw(list(range(0,100)), *OptimalParams), line=dict(color=MyColors[MyColorsIndex], width=1) ) )
                elif RegressionOptions_Dropdown.value == "Logarithmic":
                    OptimalParams, OptParamsCovariance = curve_fit(func_logarithmic, list(range(1,100)), B.JH_distribution[1:])
                    poly_str = "y = " + "{:.2e}".format(OptimalParams[0]) + " * log(x) +" + "{:.2e}".format(OptimalParams[1])
                    fig.add_trace( go.Scatter(name=B.ID+":  "+poly_str, x=list(range(1,100)), y=func_logarithmic(list(range(1,100)), *OptimalParams), line=dict(color=MyColors[MyColorsIndex], width=1) ) )
                elif RegressionOptions_Dropdown.value == "Euler":
                    OptimalParams, OptParamsCovariance = curve_fit(func_euler, list(range(1,100)), B.JH_distribution[1:])
                    poly_str = "y = " + "{:.2e}".format(OptimalParams[0]) + " / e^(" + "{:.2e}".format(OptimalParams[1]) + "*x) + " + "{:.2e}".format(OptimalParams[2])
                    fig.add_trace( go.Scatter(name=B.ID+":  "+poly_str, x=list(range(1,100)), y=func_euler(list(range(1,100)), *OptimalParams), line=dict(color=MyColors[MyColorsIndex], width=1) ) )
                elif RegressionOptions_Dropdown.value == "Maxwell":
                    OptimalParams, OptParamsCovariance = curve_fit(func_maxwellian, list(range(1,100)), B.JH_distribution[1:])
                    poly_str = "y = " + "{:.2e}".format(OptimalParams[0]) + " * x^2 * e^(-" + "{:.2e}".format(OptimalParams[1]) + "*x) + " + "{:.2e}".format(OptimalParams[2])
                    fig.add_trace( go.Scatter(name=B.ID+":  "+poly_str, x=list(range(1,100)), y=func_maxwellian(list(range(1,100)), *OptimalParams), line=dict(color=MyColors[MyColorsIndex], width=1) ) )                    

                
                # draw the distribution
                bin_desciption = B.ID + ":  " + str(B.Altitude_min) + "<Alt<"+ str(B.Altitude_max) + "  <b>" + str(B.Kp_min) + "<Kp<" + str(B.Kp_max) + "</b>" + "  Mean=" + "{:.3e}".format(B.JH_mean) + "  " + "Variance=" + "{:.3e}".format(B.JH_variance) + "  St.Deviation=" + "{:.3e}".format(B.JH_variance**(1/2))
                fig.add_trace( go.Scatter(name=bin_desciption, x=list(range(0,100)), y=B.JH_distribution, mode='markers', marker_size=3, marker_color=MyColors[MyColorsIndex]  ) )
                
                # add visuals for the mean line                
                mean_slot_idx = int(   (B.JH_mean - lower_value) / slot_length  )
                fig.add_shape( type="line", x0=mean_slot_idx, y0=0,     x1=mean_slot_idx, y1=(95/100)*max(B.JH_distribution),     line=dict( color=MyColors[MyColorsIndex], width=1, ), )    
                # add bin name above the mean line
                BinAnnotations.append( dict( x=mean_slot_idx, y=(95/100)*max(B.JH_distribution), xref="x", yref="y", text=B.ID, showarrow=False, yshift=8, font=dict(color=MyColors[MyColorsIndex])) )
                
                # add visuals for standard deviation
                #StDev_slots_width = int(   ((B.JH_variance)**(1/2)/2) / slot_length  )
                #fig.add_shape( type="line", x0=mean_slot_idx-StDev_slots_width, y0=(95/100)*max(B.JH_distribution),     x1=mean_slot_idx+StDev_slots_width, y1=(95/100)*max(B.JH_distribution),     line=dict( color=MyColors[MyColorsIndex], width=1, ), )
                
                BinIdx += 1
                
        # draw correct ticks at the x-axis, containing the JH values
        XaxisTickPositions = list()
        XaxisTickLabels = list()
        for i in range( 0, 100, 20 ):
            XaxisTickPositions.append( i )
            XaxisTickLabels.append(  "{:.3e}".format(lower_value + i*slot_length)  )            
        XaxisTickPositions.append( 99 )
        XaxisTickLabels.append(  "{:.3e}".format(upper_value)  )
        fig.update_xaxes( tickmode = 'array', tickvals=XaxisTickPositions,  ticktext=XaxisTickLabels )
                
        fig.update_layout( annotations=BinAnnotations )
        fig.update_layout( title="JH Distribution per Bin - " + CALCULATIONS_RegionName, 
                           width=1000, height=900, legend_orientation="h", legend= {'itemsizing': 'constant'}) 
        fig.update_xaxes(range=[0,99], title="Joule Heating (W/m3)", showexponent = 'all', exponentformat = 'e')
        fig.update_yaxes( title="Number of hits inside the bin") #rangemode='nonnegative'
        plotly.offline.init_notebook_mode(connected=True)
        plotly.offline.iplot(fig)


        
        
def Plot_KpScatter(OrbitFileName, from_Altitude, to_Altitude):
    print("KpScatter started for", OrbitFileName)
    maxTime = datetime.strptime( 'Mar 17 2028 00:00:03.515',  '%b %d %Y %H:%M:%S.%f')
    lineNum = 0
    with open( OrbitFileName ) as CSVfile:
        CSVreader = csv.reader( CSVfile )
        # locate the column numnbers of interest inside the csv file
        CSVheader = next( CSVreader )
        Time_idx     = CSVheader.index( "Time (UTCG)" ) #CSVheader.index( "Daedalus.EpochText" )
        Lat_idx      = CSVheader.index( "Lat (deg)" ) #CSVheader.index( "Daedalus.Latitude" )
        Lon_idx      = CSVheader.index( "Lon (deg)" ) #CSVheader.index( "Daedalus.Longitude" )
        Altitude_idx = CSVheader.index( "Alt (km)" ) #CSVheader.index( "Daedalus.Height" )
        MagLat_idx   = CSVheader.index( "Daedalus.Magnetic Latitude" )
        MLT_idx      = CSVheader.index( "Daedalus.MLT" )
        # read the satellite positions and add them to lists for ploting
        new_x_axis = list()
        new_y_axis = list()
        kp_array   = list()
        for row in CSVreader: # for each satellite position
            lineNum += 1
            CURR_Altitude = float( row[Altitude_idx] )
            if CURR_Altitude >= from_Altitude  and  CURR_Altitude <= to_Altitude:
                CURR_time = parseDaedalusDate( row[Time_idx] )
                if CURR_time == None:
                    print( "ERROR during Kp-scatter while reading", OrbitFileName, ": Wrong time format:", row[Time_idx] )
                    return 0, 0, "", 0 # <<<<
                new_x_axis.append( float( row[MLT_idx]    ) )
                new_y_axis.append( float( row[MagLat_idx] ) )                    
                kp_array.append  ( 2 )  # TODO put a correct Kp value here
                if CURR_time > maxTime: break
                
    # lessen the data so they become plottable   
    print( "WE HAVE ", len(kp_array), "points until", maxTime)
    '''
    max_num_of_points = 127000
    plot_step = int(  len(kp_array) / max_num_of_points  )
    if plot_step <= 0: plot_step = 1
    print( "I will plot", max_num_of_points, "out of", len(kp_array), "points (1 per", plot_step, ")")
    new_x_axis = new_x_axis[0:max_num_of_points:2] #new_x_axis = new_x_axis[0::plot_step]
    new_y_axis = new_y_axis[0:max_num_of_points:2] # new_y_axis = new_y_axis[0::plot_step]
    kp_array   = kp_array[0:max_num_of_points:2] #kp_array   = kp_array[0::plot_step]
    '''
                
    # create the Kp scatter     
    fig = go.Figure()
    fig.add_trace(go.Scatter( x=new_x_axis, y=new_y_axis,
                              mode='markers', marker=dict(color=kp_array, cmin=0, cmax=9, size=3, colorscale='rainbow',showscale=True, 
                                                          colorbar=dict(title="Kp", xanchor="left", x=-0.26, tick0=0, dtick=1, tickvals=[0,1,2,3,4,5,6,7,8,9], ticks="inside") ) ))
    fig.update_layout( width=1200, height=800, showlegend=False, coloraxis_showscale=False, title="Orbit file:" + OrbitFileName + "<br>" + "<b>Kp Indices during Mission Lifetime for Altitudes: "+str(from_Altitude)+"-"+str(to_Altitude)+" km</b>")
    fig.update_xaxes(title="Magnetic Local Time", showgrid=True, gridwidth=0.5, gridcolor='grey')
    fig.update_layout(xaxis = dict(tickmode = 'linear',tick0 = 0,dtick = 2))    
    fig.update_yaxes(title="Magnetic Latitude (deg)", showgrid=True, gridwidth=0.5, gridcolor='gray') 
    fig.update_layout(yaxis = dict(tickmode = 'linear',tick0 = -90,dtick =30), margin=go.layout.Margin(b=150,t=150), width=1000, height=800, showlegend=True)
    fig.update_xaxes(range=[0, 24],  showline=True, linewidth=2, linecolor='gray', mirror=True)
    fig.update_yaxes(range=[-90, 90],showline=True, linewidth=2, linecolor='gray', mirror=True)
    # draw rectangles to represent the bins on the figure        
    MyColorsIndex = 0
    for B in Bins:
        if B.Altitude_min>=from_Altitude and B.Altitude_max<=to_Altitude:
            BinInfo = "{:3.0f}".format(B.Altitude_min) + "-" + "{:3.0f}".format(B.Altitude_max) + "km " + ": Kp"+ str(B.Kp_min) + "-" + str(B.Kp_max) + ": " 
            if B.CumulativeTime > B.DesirableCumulativeTime: BinInfo += "<b>"
            BinInfo += "{:3.0f}".format(B.CumulativeTime/60) 
            if B.CumulativeTime > B.DesirableCumulativeTime: BinInfo += "</b>"
            BinInfo += "/" + "{:3.0f}".format(B.DesirableCumulativeTime/60)  + "min"
            if B.MLT_min <= B.MLT_max: # ex: 10:00-14:00
                Xmin1 = B.MLT_min
                Xmax1 = B.MLT_max
                Xmin2 = -1
                Xmax2 = -1
            else: # ex: 22:00-02:00
                Xmin1 = B.MLT_min
                Xmax1 = 24
                Xmin2 = 0
                Xmax2 = B.MLT_max
            # check if this rectangle has been already plotted
            FoundFigureIndex = -1 
            for i in range( len(fig.layout['shapes']) ):
                if fig.layout['shapes'][i]["x0"]==Xmin1 and fig.layout['shapes'][i]["y0"]==B.MagLat_min and fig.layout['shapes'][i]["x1"]==Xmax1 and fig.layout['shapes'][i]["y1"]==B.MagLat_max:
                    FoundFigureIndex = i
                    break
            # add info about this bin to the legend text and/or draw a new rectangle
            if FoundFigureIndex >= 0: 
                for i in range( len(fig['data']) ):
                    if fig['data'][i]['name'] is not None  and   B.Description in fig['data'][i]['name']:
                        fig['data'][i]['name'] += BinInfo + "<br>"
            else:
                # draw a rectangle representing the bin
                fig.add_shape(go.layout.Shape( type="rect", xref="x", yref="y", opacity=0.7, layer="above",
                    x0=Xmin1, y0=B.MagLat_min, x1=Xmax1, y1=B.MagLat_max, 
                    line=dict(color=MyColors[MyColorsIndex], width=2,), fillcolor=MyColors[MyColorsIndex], 
                ))
                # add a trace so that a legend about the rectangle is created
                fig.add_trace(go.Scatter( x=[0], y=[0], marker=dict(color=MyColors[MyColorsIndex], size=0, opacity=0), 
                              name="<b>" + B.Description + "</b>" + "<br>" + BinInfo + "<br>" ))
                # draw a 2nd rectangle if necessary for this bin (when for ex it is 22:00-02:00)
                if Xmin1>=0 and Xmin2>=0:
                    fig.add_shape(go.layout.Shape( type="rect", xref="x", yref="y", opacity=0.7, layer="above",
                        x0=Xmin2, y0=B.MagLat_min, x1=Xmax2, y1=B.MagLat_max, 
                        line=dict(color=MyColors[MyColorsIndex], width=2,), fillcolor=MyColors[MyColorsIndex],
                    ))
                MyColorsIndex += 1
                
    # plot it
    plotly.offline.init_notebook_mode(connected=True)
    plotly.offline.iplot(fig) 
    print("KpScatter finished")
        

        
        
def plotComparisonOfResults():
    # ColorTriplet = ["#063852", "#F0810F", "#E6DF44"] no
    #ColorTriplet = ["#F8F1E5", "#426E86", "#F9BA32"] no - blue yellow
    #ColorTriplet = ["#FFFFFF", "#FA8D62", "#2B616D"] no
    ColorTriplet = ["#EEEEEE", "#FF4447", "#257985"] # yes - white red petrol - needs a darker bg
    #ColorTriplet = ["#E2C499", "#C8000A", "#E8A735"] no
    #ColorTriplet = ["#D6C6B9", "#97B8C2", "#D35C37"] no
    #ColorTriplet = ["#CDBEA7", "#882426", "#C29545"] no
    #ColorTriplet = ["#A3A599", "#FBCD4B", "#88A550"] # yes - gray green yellow - needs a lighter bg
    #ColorTriplet = ["#81715E", "#FAAE3D", "#E4535E"] no
    #ColorTriplet = ["#FFB74C", "#FD3C3C", "#138D90"] # yes - ochra red blue - needs a lighter bg
    #ColorTriplet = ["#DDDEDE", "#A5C05B", "#7BA4A8"] no
    # init
    BinIDs = list()
    Means1 = list()
    Means2 = list()
    StDev1 = list()
    StDev2 = list()
    RegionDescription = ""
    # load results no1
    LoadResults( SavedFilenames_Dropdown.value )
    for B in Bins:
        if B.JH_mean != 0:
            RegionDescription = B.Description
            BinIDs.append( B.ID )
            Means1.append( B.JH_mean )
            StDev1.append( math.sqrt(B.JH_variance) )
            print( B.ID, B.JH_mean, math.sqrt(B.JH_variance), B.JH_variance**(1/2) )
    # load results no2
    LoadResults( SavedFilenames2_Dropdown.value )
    for B in Bins:
        if B.JH_mean != 0:
            Means2.append( B.JH_mean )
            StDev2.append( math.sqrt(B.JH_variance) )
    # plot bars chart
    Bars = list()
    fig = go.Figure(data=[
        go.Bar(name='TIEGCM - JH Mean', x=BinIDs, y=Means1, marker_color=ColorTriplet[1], offsetgroup=0),
        go.Bar(name='Orbit  - JH Mean', x=BinIDs, y=Means2, marker_color=ColorTriplet[2], offsetgroup=1),
        #go.Bar(name='JH StDv 1', x=BinIDs, y=StDev1, marker_color="red",  offsetgroup=0, base=Means1),
        #go.Bar(name='JH StDv 2', x=BinIDs, y=StDev2, marker_color="cyan", offsetgroup=1, base=Means2)
    ])
    fig.update_layout(barmode='group', title='Statistics Comparison - '+RegionDescription , plot_bgcolor=ColorTriplet[0], yaxis = dict(showexponent = 'all',exponentformat = 'e'))
    plotly.offline.init_notebook_mode(connected=True)
    plotly.offline.iplot(fig) 
    print("Comparison plots finished.")

    
    
    
    
    
    
display( createGUI() )
Plot_JHvsMagLat_Checkbox.value = False
Plot_JHvsMLT_Checkbox.value = False
Plot_JHvsAltitude_Checkbox.value = False
Plot_AltitudeVsMagLat_Checkbox.value = False





Joule-Heating-per-Bin calculation started. 2020-05-08 21:43:22.188521
85
['/home/NAS/TIEGCM_DATA/TIEGCM_Lifetime_2015_to_2018_JH_QD/TIEGCM_2015/tiegcm2.0_res2.5_3years_sech_002_JH_QD_AllVars.nc', '/home/NAS/TIEGCM_DATA/TIEGCM_Lifetime_2015_to_2018_JH_QD/TIEGCM_2015/tiegcm2.0_res2.5_3years_sech_003_JH_QD_AllVars.nc', '/home/NAS/TIEGCM_DATA/TIEGCM_Lifetime_2015_to_2018_JH_QD/TIEGCM_2015/tiegcm2.0_res2.5_3years_sech_004_JH_QD_AllVars.nc', '/home/NAS/TIEGCM_DATA/TIEGCM_Lifetime_2015_to_2018_JH_QD/TIEGCM_2015/tiegcm2.0_res2.5_3years_sech_005_JH_QD_AllVars.nc', '/home/NAS/TIEGCM_DATA/TIEGCM_Lifetime_2015_to_2018_JH_QD/TIEGCM_2015/tiegcm2.0_res2.5_3years_sech_006_JH_QD_AllVars.nc', '/home/NAS/TIEGCM_DATA/TIEGCM_Lifetime_2015_to_2018_JH_QD/TIEGCM_2015/tiegcm2.0_res2.5_3years_sech_007_JH_QD_AllVars.nc', '/home/NAS/TIEGCM_DATA/TIEGCM_Lifetime_2015_to_2018_JH_QD/TIEGCM_2015/tiegcm2.0_res2.5_3years_sech_008_JH_QD_AllVars.nc', '/home/NAS/TIEGCM_DATA/TIEGCM_Lifetime_2015_to_2018_JH_QD/TIEGCM_2015/ti